In [ ]:
# === DELFIN Dashboard ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
import json
import json
import py3Dmol
import os
import subprocess
import shutil
import html
import re
import base64
import sys
import numpy as np
from pathlib import Path

# RDKit für SMILES-Konvertierung
from rdkit import Chem
from rdkit.Chem import AllChem

# stk für Metallkomplexe (optional)
try:
    import stk
    HAS_STK = True
except ImportError:
    HAS_STK = False

# Ensure local DELFIN is importable in notebooks
_deflin_root = None
for _base in [Path.cwd(), *Path.cwd().parents]:
    if (_base / 'software' / 'delfin' / 'delfin').is_dir():
        _deflin_root = _base / 'software' / 'delfin'
        break
    if (_base / 'delfin').is_dir():
        _deflin_root = _base
        break
if _deflin_root and str(_deflin_root) not in sys.path:
    sys.path.insert(0, str(_deflin_root))

# Force reload to pick up any code changes
import importlib
import delfin.config
import delfin.common.control_validator
importlib.reload(delfin.common.control_validator)
importlib.reload(delfin.config)
from delfin.config import validate_control_text

# Pfade dynamisch ermitteln
HOME = Path.home()

# Notebook-Verzeichnis ermitteln (wo dieses Notebook liegt)
try:
    import IPython
    NOTEBOOK_DIR = Path(IPython.extract_module_locals()[1]['__vsc_ipynb_file__']).parent
except:
    NOTEBOOK_DIR = Path.cwd()

def find_root_dir(start_dir: Path):
    cur = start_dir
    while True:
        if (cur / "software" / "delfin").is_dir():
            return cur
        if cur == cur.parent:
            return None
        cur = cur.parent

ROOT_DIR = find_root_dir(NOTEBOOK_DIR)
if ROOT_DIR is None:
    ROOT_DIR = HOME

BASE_DIR = ROOT_DIR / "calc"
CALC_DIR = BASE_DIR  # Store reference to calc folder for Calculations tab

_submit_candidates = [
    ROOT_DIR / "software" / "delfin" / "examples" / "example_Job_Submission_Scripts" / "BwUniCluster" / "submit_sh",
    NOTEBOOK_DIR / "submit_sh",
]
SUBMIT_TEMPLATES_DIR = None
for _path in _submit_candidates:
    if _path.exists():
        SUBMIT_TEMPLATES_DIR = _path
        break
if SUBMIT_TEMPLATES_DIR is None:
    SUBMIT_TEMPLATES_DIR = _submit_candidates[0]

_orca_candidates = []
try:
    _orca_root = ROOT_DIR / "software"
    if _orca_root.exists():
        for _p in sorted(_orca_root.glob("orca_*")):
            if _p.is_dir():
                _orca_candidates.append(_p)
except Exception:
    _orca_candidates = []

ORCA_BASE_SELECTED = None
if _orca_candidates:
    for _p in _orca_candidates:
        if "6_1_1" in _p.name:
            ORCA_BASE_SELECTED = str(_p)
            break
    if ORCA_BASE_SELECTED is None:
        ORCA_BASE_SELECTED = str(_orca_candidates[-1])


_goat_candidates = [
    ROOT_DIR / "software" / "CONTROL_Templates" / "only_GOAT.txt",
    ROOT_DIR / "CONTROL_Templates" / "only_GOAT.txt",
    NOTEBOOK_DIR / "CONTROL_Templates" / "only_GOAT.txt",
]
ONLY_GOAT_TEMPLATE_PATH = None
for _path in _goat_candidates:
    if _path.exists():
        ONLY_GOAT_TEMPLATE_PATH = _path
        break
if ONLY_GOAT_TEMPLATE_PATH is None:
    ONLY_GOAT_TEMPLATE_PATH = _goat_candidates[0]

JOB_TIME_LIMITS = {
    '12h': '12:00:00',
    '24h': '24:00:00',
    '36h': '36:00:00',
    '48h': '48:00:00',
    '60h': '60:00:00',
    '72h': '72:00:00'
}


def submit_job(job_dir, mode, job_name, inp_file=None, time_limit='48:00:00', ntasks=40, mem_mb=240000):
    """
    Central submit function using environment variables.
    
    Args:
        job_dir: Directory to submit from
        mode: 'delfin' | 'delfin-recalc' | 'orca' | 'auto'
        job_name: Name for the job
        inp_file: Specific .inp file for ORCA mode (optional)
        time_limit: SLURM time limit (e.g., '48:00:00')
        ntasks: Number of CPUs
        mem_mb: Memory in MB
    
    Returns:
        subprocess.CompletedProcess result
    """
    env_vars = f"DELFIN_MODE={mode},DELFIN_JOB_NAME={job_name}"
    if ORCA_BASE_SELECTED:
        env_vars += f",DELFIN_ORCA_BASE={ORCA_BASE_SELECTED}"
    if inp_file:
        env_vars += f",DELFIN_INP_FILE={inp_file}"

    # Prefer PAL/maxcore from CONTROL.txt, fallback to .inp
    pal_used = None
    maxcore_used = None
    try:
        control_path = Path(job_dir) / 'CONTROL.txt'
        if control_path.exists():
            pal_used, maxcore_used = parse_resource_settings(control_path.read_text())
    except Exception:
        pal_used, maxcore_used = None, None

    if pal_used is None or maxcore_used is None:
        try:
            inp_candidate = None
            if inp_file:
                cand = Path(job_dir) / inp_file
                if cand.exists():
                    inp_candidate = cand
            if inp_candidate is None:
                inp_files = sorted(Path(job_dir).glob('*.inp'))
                inp_candidate = inp_files[0] if inp_files else None
            if inp_candidate is not None and inp_candidate.exists():
                pal_inp, maxcore_inp = parse_inp_resources(inp_candidate.read_text())
                if pal_used is None:
                    pal_used = pal_inp
                if maxcore_used is None:
                    maxcore_used = maxcore_inp
        except Exception:
            pass

    if pal_used is None:
        pal_used = int(ntasks)
    mem_used = int(mem_mb)
    if pal_used and maxcore_used:
        mem_used = int(pal_used) * int(maxcore_used)

    return subprocess.run([
        'sbatch',
        f'--export=ALL,{env_vars}',
        f'--time={time_limit}',
        '--ntasks=1',
        f'--cpus-per-task={pal_used}',
        f'--mem={mem_used}M',
        f'--job-name={job_name}',
        str(SUBMIT_TEMPLATES_DIR / 'submit_delfin.sh')
    ], cwd=job_dir, capture_output=True, text=True)

# === SMILES Konvertierungsfunktionen ===

def contains_metal(smiles):
    """Prüfe ob SMILES ein Metall enthält."""
    metals = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Be', 'Mg', 'Ca', 'Sr', 'Ba',
              'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
              'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
              'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy',
              'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os',
              'Ir', 'Pt', 'Au', 'Hg', 'Al', 'Ga', 'In', 'Tl', 'Sn', 'Pb',
              'Bi', 'Po', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu']
    for metal in metals:
        if re.search(rf'\[{metal}[+\-\d\]@]', smiles, re.IGNORECASE):
            return True
        if re.search(rf'\[{metal}\]', smiles, re.IGNORECASE):
            return True
    return False




def mol_from_smiles_rdkit(smiles, allow_metal=False):
    """Robuste RDKit-SMILES-Erzeugung, toleranter bei Metallkomplexen."""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return mol, None
        if not allow_metal:
            return None, "Ungültiger SMILES"
        mol = Chem.MolFromSmiles(smiles, sanitize=False)
        if mol is None:
            return None, "Ungültiger SMILES (ohne Sanitizing)"
        try:
            Chem.SanitizeMol(
                mol,
                sanitizeOps=(
                    Chem.SanitizeFlags.SANITIZE_ALL
                    ^ Chem.SanitizeFlags.SANITIZE_PROPERTIES
                    ^ Chem.SanitizeFlags.SANITIZE_KEKULIZE
                ),
            )
            return mol, "partielles Sanitizing"
        except Exception:
            return mol, "Sanitizing übersprungen"
    except Exception as e:
        return None, f"RDKit Fehler: {e}"
def smiles_to_xyz(smiles):
    """
    Konvertiere SMILES zu XYZ-Koordinaten (ohne Optimierung).
    Verwendet stk für Metallkomplexe, RDKit für organische Moleküle.
    Returns: (xyz_string, num_atoms, method_used, error_msg)
    """
    has_metal = contains_metal(smiles)
    stk_error = None
    
    # Versuche stk für Metallkomplexe
    if has_metal and HAS_STK:
        try:
            bb = stk.BuildingBlock(smiles)
            mol = bb.to_rdkit_mol()
            if mol.GetNumConformers() == 0:
                AllChem.EmbedMolecule(mol, randomSeed=42, useRandomCoords=True)
            method = "stk"
        except Exception as e:
            # Fallback zu RDKit
            stk_error = str(e)
            mol = None
            method = None
    else:
        mol = None
        method = None
    
    # RDKit als Fallback oder für organische Moleküle
    if mol is None:
        mol, rdkit_note = mol_from_smiles_rdkit(smiles, allow_metal=has_metal)
        if mol is None:
            if stk_error:
                return None, 0, None, f"RDKit: {rdkit_note}; stk: {stk_error}"
            return None, 0, None, rdkit_note
        
        try:
            mol = Chem.AddHs(mol)
        except Exception:
            pass

        try:
            params = AllChem.ETKDGv3()
            params.randomSeed = 42
            params.useRandomCoords = True
            try:
                params.maxAttempts = 100
            except Exception:
                pass

            try:
                result = AllChem.EmbedMolecule(mol, params, maxAttempts=50)
            except TypeError:
                result = AllChem.EmbedMolecule(mol, params)
            if result == -1:
                params2 = AllChem.ETKDGv3()
                params2.randomSeed = 42
                params2.useRandomCoords = True
                try:
                    params2.maxAttempts = 200
                except Exception:
                    pass
                try:
                    result = AllChem.EmbedMolecule(mol, params2, maxAttempts=200)
                except TypeError:
                    result = AllChem.EmbedMolecule(mol, params2)
                if result == -1:
                    return None, 0, None, "Konnte keine 3D-Struktur generieren"
            method = "RDKit"
            if rdkit_note:
                method = f"RDKit ({rdkit_note})"
        except Exception as e:
            return None, 0, None, f"RDKit Fehler: {str(e)}"
    
    # XYZ-String erstellen
    try:
        conf = mol.GetConformer()
        num_atoms = mol.GetNumAtoms()
        xyz_lines = []
        for i in range(num_atoms):
            atom = mol.GetAtomWithIdx(i)
            pos = conf.GetAtomPosition(i)
            xyz_lines.append(f"{atom.GetSymbol():2s} {pos.x:12.6f} {pos.y:12.6f} {pos.z:12.6f}")
        
        xyz_string = '\n'.join(xyz_lines)
        return xyz_string, num_atoms, method, None
    except Exception as e:
        return None, 0, None, f"Koordinaten-Fehler: {str(e)}"


def is_smiles(text):
    """Prueft ob der Text ein SMILES-String ist."""
    text = text.strip()
    if '\n' in text:
        return False
    smiles_pattern = r'^[A-Za-z0-9\[\]()=#@+\-\\\/.:,%]+$'
    return bool(re.match(smiles_pattern, text))


def clean_input_data(input_text):
    """
    Verarbeitet Eingabedaten: SMILES oder XYZ-Koordinaten.
    Returns: (cleaned_text, input_type)
    """
    text = input_text.strip()
    
    if not text:
        return '', 'empty'
    
    if is_smiles(text):
        return text, 'smiles'
    
    lines = text.split('\n')
    if len(lines) < 2:
        return text, 'xyz'
    
    first_line = lines[0].strip()
    try:
        atom_count = int(first_line)
        cleaned_lines = lines[2:]
        return '\n'.join(cleaned_lines).strip(), 'xyz'
    except ValueError:
        return text, 'xyz'


def parse_resource_settings(control_text):
    """Parse PAL and maxcore from CONTROL.txt content."""
    pal_match = re.search(r'^\s*PAL\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    maxcore_match = re.search(r'^\s*maxcore\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    pal = int(pal_match.group(1)) if pal_match else None
    maxcore = int(maxcore_match.group(1)) if maxcore_match else None
    return pal, maxcore


DEFAULT_CONTROL = """input_file=input.txt
NAME=
SMILES=
charge=[CHARGE]
------------------------------------
Solvation:
implicit_solvation_model=CPCM
solvent=[SOLVENT]
XTB_SOLVATOR=no
number_explicit_solv_molecules=2
------------------------------------
Global geometry optimisation:
xTB_method=XTB2
XTB_OPT=no
XTB_GOAT=no
CREST=no
multiplicity_global_opt=
------------------------------------
IMAG=yes
IMAG_scope=initial
IMAG_option=2
allow_imaginary_freq=0
IMAG_sp_energy_window=1e-3
IMAG_optimize_candidates=no
------------------------------------
calc_prop_of_interest=no
properties_of_interest=IP,EA
------------------------------------
Redox steps:
calc_initial=yes
oxidation_steps=1,2,3
reduction_steps=1,2,3
method=classic|manually|OCCUPIER
calc_potential_method=2
------------------------------------
ESD module (excited state dynamics):
ESD_modul=no
ESD_modus=TDDFT|deltaSCF|hybrid1
ESD_frequency=yes
states=S1,T1,S2,T2
ISCs=S1>T1,T1>S1
ICs=S2>S1
emission_rates=f,p
phosp_IROOT=1,2,3
phosp_keywords=
fluor_keywords=
TROOTSSL=-1,0,1
addition_S0=
DOHT=TRUE
ESD_LINES=LORENTZ
ESD_LINEW=50
ESD_INLINEW=250
ESD_NPOINTS=131072
ESD_MAXTIME=12000
hybrid1_geom_MaxIter=60
--------------------
Electrical Properties:
elprop_Dipole=no
elprop_Quadrupole=no
elprop_Hyperpol=no
elprop_Polar=no
elprop_PolarVelocity=no
elprop_PolarDipQuad=no
elprop_PolarQuadQuad=no
--------------------
deltaSCF Settings:
deltaSCF_DOMOM=true
deltaSCF_PMOM=false
deltaSCF_keepinitialref=true
deltaSCF_SOSCFHESSUP=LSR1
deltaSCF_keywords=FreezeAndRelease
deltaSCF_maxiter=300
deltaSCF_SOSCFConvFactor=500
deltaSCF_SOSCFMaxStep=0.1
--------------------
TDDFT Settings:
TDDFT_TDDFT_maxiter=500
TDDFT_nroots=15
TDDFT_maxdim=30
TDDFT_TDA=FALSE
TDDFT_followiroot=true
TDDFT_SOC=false
------------------------------------
MANUALLY:
multiplicity_0=
additions_0=
additions_TDDFT=
additions_T1=
additions_S1=
multiplicity_ox1=
additions_ox1=
multiplicity_ox2=
additions_ox2=
multiplicity_ox3=
additions_ox3=
multiplicity_red1=
additions_red1=
multiplicity_red2=
additions_red2=
multiplicity_red3=
additions_red3=
------------------------------------
Level of Theory:
functional=PBE0
disp_corr=D4
ri_jkx=RIJCOSX
relativity=ZORA
aux_jk=def2/J
aux_jk_rel=SARC/J
main_basisset=def2-SVP
main_basisset_rel=ZORA-def2-SVP
metal_basisset=def2-TZVP
metal_basisset_rel=SARC-ZORA-TZVP
first_coordination_sphere_metal_basisset=no
first_coordination_sphere_scale=1.3
geom_opt=OPT
freq_type=FREQ
initial_guess=PModel
temperature=298.15
maxiter=125
qmmm_option=QM/PBEH-3c
------------------------------------
Reference value:
E_ref=
------------------------------------
Literature_reference=
reference_CV=V Vs. Fc+/Fc
E_00_exp=
E_red_exp=
E_red_2_exp=
E_red_3_exp=
E_ox_exp=
E_ox_2_exp=
E_ox_3_exp=
*E_red_exp=
*E_ox_exp=
------------------------------------
Prints:
print_MOs=yes
print_Loewdin_population_analysis=no
------------------------------------
Resource Settings:
PAL=40
maxcore=6000
parallel_workflows=yes
pal_jobs=4
orca_parallel_strategy=auto
enable_job_timeouts=no
job_timeout_hours=36
opt_timeout_hours=14
frequency_timeout_hours=36
sp_timeout_hours=3
------------------------------------
Automatic Error Recovery & Retry:
enable_auto_recovery=yes
max_recovery_attempts=3
enable_adaptive_parallelism=yes
enable_performance_metrics=yes
------------------------------------
OCCUPIER-Settings:
--------------------
OCCUPIER_method=auto
OCCUPIER_tree=own
OWN_TREE_PURE_WINDOW=3
OWN_progressive_from=no
fob_equal_weights=yes
frequency_calculation_OCCUPIER=no
occupier_selection=tolerance
occupier_precision=3
occupier_epsilon=5e-4
clean_override_window_h=0.002
clean_quality_improvement=0.05
clean_quality_good=0.05
maxiter_occupier=125
geom_opt_OCCUPIER=OPT
pass_wavefunction=no
approximate_spin_projection_APMethod=2
--------------------
OCCUPIER_sequence_profiles:
-3,-2,-1,0,+1,+2,+3=[
even electron number:
even_seq = [
  {"index": 1, "m": 1, "BS": "",    "from": 0},
  {"index": 2, "m": 1, "BS": "1,1", "from": 1},
  {"index": 3, "m": 1, "BS": "2,2", "from": 2},
  {"index": 4, "m": 3, "BS": "",    "from": 1},
  {"index": 5, "m": 3, "BS": "3,1", "from": 4},
  {"index": 6, "m": 3, "BS": "4,2", "from": 5},
  {"index": 7, "m": 5, "BS": "",    "from": 4}
]
-------------------
odd electron number:
odd_seq = [
  {"index": 1, "m": 2, "BS": "",    "from": 0},
  {"index": 2, "m": 2, "BS": "2,1", "from": 1},
  {"index": 3, "m": 2, "BS": "3,2", "from": 2},
  {"index": 4, "m": 4, "BS": "",    "from": 1},
  {"index": 5, "m": 4, "BS": "4,1", "from": 4},
  {"index": 6, "m": 4, "BS": "5,2", "from": 5},
  {"index": 7, "m": 6, "BS": "",    "from": 4}
]
]
"""

common_layout = widgets.Layout(width='500px')
common_style = {'description_width': 'initial'}

# === Widgets ===
job_name_widget = widgets.Text(
    value='',
    placeholder='e.g. Fe_Complex_Ox',
    description='Job Name:',
    layout=common_layout,
    style=common_style
)

job_type_widget = widgets.ToggleButtons(
    options=['12h', '24h', '36h', '48h', '60h', '72h'],
    value='48h',
    description='Time Limit:',
    style=common_style,
    button_style='',
    layout=widgets.Layout(width='550px')
)

coords_help = widgets.Label("Input: XYZ coordinates (header auto-removed) or SMILES string")
coords_widget = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates or SMILES:\n\nXYZ example:\n42\nComment\nFe  0.0  0.0  0.0\nC   1.5  0.0  0.0\n\nSMILES example:\nCCO or c1ccccc1',
    description='Input:',
    layout=widgets.Layout(width='500px', height='200px'),
    style=common_style
)

# Spacer für konsistenten Abstand vor Buttons (10px)
button_spacer = widgets.Label(value="", layout=widgets.Layout(height="10px"))

convert_smiles_button = widgets.Button(
    description='CONVERT SMILES',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

smiles_batch_help = widgets.Label("Batch SMILES list: one per line 'Name;SMILES;key=value;key=value'")
smiles_batch_widget = widgets.Textarea(
    value='',
    placeholder='name;SMILES;key=value;...\nNi_1;[Ni];charge=2;solvent=water\nCo_1;[Co];charge=3',
    description='SMILES List:',
    layout=widgets.Layout(width='500px', height='160px'),
    style=common_style
)

submit_smiles_list_button = widgets.Button(
    description='SUBMIT SMILES LIST',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
smiles_batch_output = widgets.Output()

control_help = widgets.Label("CONTROL.txt - edit parameters as needed")
control_widget = widgets.Textarea(
    value=DEFAULT_CONTROL,
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='500px'),
    style=common_style
)

submit_button = widgets.Button(
    description='SUBMIT JOB',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

validate_button = widgets.Button(
    description='VALIDATE CONTROL',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

output_area = widgets.Output()
validate_output = widgets.Output()

mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #1976d2',
    width='560px',
    height='420px',
    overflow='hidden'
))

xyz_copy_btn = widgets.Button(
    description='📋 Copy Coordinates',
    button_style='success',
    layout=widgets.Layout(width='150px'),
    disabled=True
)
xyz_copy_status = widgets.HTML(value='', layout=widgets.Layout(margin='0 0 0 6px'))
xyz_copy_row = widgets.HBox([
    xyz_copy_btn,
    xyz_copy_status
], layout=widgets.Layout(gap='6px', align_items='center'))


only_goat_label = widgets.Label("Only GOAT:")
only_goat_charge = widgets.IntText(
    value=0,
    description='Charge:',
    style=common_style,
    layout=widgets.Layout(width='160px')
)
only_goat_solvent = widgets.Text(
    value='',
    placeholder='e.g. water',
    description='Solvent:',
    style=common_style,
    layout=widgets.Layout(width='220px')
)
only_goat_submit_button = widgets.Button(
    description='SUBMIT ONLY GOAT',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
only_goat_output = widgets.Output()

# Speichere konvertierte XYZ-Koordinaten für Job-Submit
_converted_xyz_cache = {'smiles': None, 'xyz': None}
# Aktuelle XYZ-Koordinaten für Copy-Button
_current_xyz_for_copy = {'content': None}


def update_molecule_view(change=None):
    """Update Molekül-Visualisierung - konvertiert SMILES nicht automatisch."""
    global _converted_xyz_cache, _current_xyz_for_copy

    with mol_output:
        clear_output()
        raw_input = coords_widget.value.strip()

        if not raw_input:
            print("Please enter XYZ coordinates or SMILES.")
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            _current_xyz_for_copy = {'content': None}
            xyz_copy_btn.disabled = True
            xyz_copy_status.value = ''
            return

        cleaned_data, input_type = clean_input_data(raw_input)

        if input_type == 'smiles':
            # Avoid heavy conversion on every keystroke; use the convert button instead.
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            _current_xyz_for_copy = {'content': None}
            xyz_copy_btn.disabled = True
            xyz_copy_status.value = ''
            print("SMILES erkannt. Bitte 'CONVERT SMILES' klicken.")
            return

        # XYZ-Koordinaten direkt visualisieren
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        coords = cleaned_data
        lines = [l for l in coords.split('\n') if l.strip()]
        num_atoms = len(lines)
        xyz_data = f"{num_atoms}\nGenerated by widget\n{coords}"
        _current_xyz_for_copy = {'content': xyz_data}
        xyz_copy_btn.disabled = False
        xyz_copy_status.value = '<span style="color:#388e3c;">XYZ ready to copy</span>'
        mol_view = py3Dmol.view(width=560, height=420)
        mol_view.addModel(xyz_data, "xyz")
        mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
        mol_view.zoomTo()
        mol_view.show()


def on_xyz_copy(button):
    content = _current_xyz_for_copy.get('content') if _current_xyz_for_copy else None
    if not content:
        xyz_copy_status.value = '<span style="color:#d32f2f;">No XYZ to copy</span>'
        return
    escaped = content.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code = (
        "navigator.clipboard.writeText('" + escaped + "')"
        ".then(() => console.log('Copied XYZ'))"
        ".catch(err => console.error('Copy failed:', err));"
    )
    display(Javascript(js_code))
    xyz_copy_status.value = '<span style="color:#388e3c;">Copied to clipboard</span>'


xyz_copy_btn.on_click(on_xyz_copy)
coords_widget.observe(update_molecule_view, names='value')


def handle_convert_smiles(button):
    global _converted_xyz_cache
    raw_input = coords_widget.value.strip()
    if not raw_input:
        with mol_output:
            clear_output()
            print("Please enter SMILES in the input box.")
        return

    with mol_output:
        clear_output()
        print("Converting SMILES...")

    cleaned_data, input_type = clean_input_data(raw_input)
    if input_type != 'smiles':
        with mol_output:
            clear_output()
            print("Input is not a SMILES string.")
        return

    xyz_string, num_atoms, method, error = smiles_to_xyz(cleaned_data)
    if error:
        with mol_output:
            clear_output()
            print(f"SMILES: {cleaned_data}")
            print(f"Fehler: {error}")
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        return

    # Cache SMILES and XYZ for later use in submit
    _converted_xyz_cache = {'smiles': cleaned_data, 'xyz': xyz_string}
    coords_widget.value = f"{num_atoms}\nConverted from SMILES ({method})\n{xyz_string}"


convert_smiles_button.on_click(handle_convert_smiles)

def handle_submit_smiles_list(button):
    with smiles_batch_output:
        clear_output()

        job_prefix = job_name_widget.value.strip()
        if not job_prefix:
            print("Error: Job name cannot be empty!")
            return

        control_content_base = control_widget.value
        control_errors = validate_control_text(control_content_base)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return

        job_type = job_type_widget.value
        time_limit = JOB_TIME_LIMITS.get(job_type, '48:00:00')

        entries = [l.strip() for l in smiles_batch_widget.value.splitlines() if l.strip()]
        if not entries:
            print("Error: SMILES list is empty.")
            return

        for idx, entry in enumerate(entries, 1):
            if ';' not in entry:
                print(f"Line {idx}: Missing ';' delimiter -> {entry}")
                continue
            parts = [p.strip() for p in entry.split(';') if p.strip()]
            if len(parts) < 2:
                print(f"Line {idx}: Missing name or SMILES -> {entry}")
                continue

            name_raw = parts[0]
            smiles = parts[1]
            extra_parts = parts[2:]

            if not name_raw or not smiles:
                print(f"Line {idx}: Missing name or SMILES -> {entry}")
                continue

            extras = {}
            for part in extra_parts:
                if '=' not in part:
                    print(f"Line {idx}: Invalid override '{part}' (expected key=value)")
                    continue
                key, value = part.split('=', 1)
                key = key.strip()
                value = value.strip()
                if not key:
                    print(f"Line {idx}: Invalid override '{part}' (empty key)")
                    continue
                extras[key] = value

            safe_name = ''.join(c for c in name_raw if c.isalnum() or c in ('_', '-'))
            if not safe_name:
                print(f"Line {idx}: Invalid name -> {name_raw}")
                continue

            job_name = f"{job_prefix}_{safe_name}"
            safe_job_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
            if not safe_job_name:
                print(f"Line {idx}: Invalid job name -> {job_name}")
                continue

            xyz_string, num_atoms, method, error = smiles_to_xyz(smiles)
            if error:
                print(f"Line {idx}: {safe_name} - SMILES error: {error}")
                continue

            job_dir = BASE_DIR / safe_job_name
            job_dir.mkdir(parents=True, exist_ok=True)

            control_content = re.sub(r'(?m)^SMILES=.*$', f"SMILES={smiles}", control_content_base)
            for key, value in extras.items():
                pattern = rf'(?m)^{re.escape(key)}\s*=.*$'
                replacement = f"{key}={value}"
                if re.search(pattern, control_content):
                    control_content = re.sub(pattern, replacement, control_content)
                else:
                    control_content = control_content.rstrip() + f"\n{replacement}\n"

            control_path = job_dir / 'CONTROL.txt'
            control_path.write_text(control_content)

            input_path = job_dir / 'input.txt'
            input_path.write_text(xyz_string)

            pal, maxcore = parse_resource_settings(control_content)
            total_mem = pal * maxcore if pal and maxcore else 240000

            result = submit_job(
                job_dir=job_dir,
                mode='delfin',
                job_name=safe_job_name,
                time_limit=time_limit,
                ntasks=pal or 40,
                mem_mb=total_mem
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '(unknown)'
                print(f"Submitted {safe_job_name} (ID: {job_id})")
            else:
                print(f"Failed {safe_job_name}: {result.stderr or result.stdout}")


submit_smiles_list_button.on_click(handle_submit_smiles_list)


def reset_form():
    """Setzt das Formular zurueck nach erfolgreichem Submit."""
    global _converted_xyz_cache
    job_name_widget.value = ''
    coords_widget.value = ''
    smiles_batch_widget.value = ''
    control_widget.value = DEFAULT_CONTROL
    job_type_widget.value = '48h'
    only_goat_charge.value = 0
    only_goat_solvent.value = ''
    _converted_xyz_cache = {'smiles': None, 'xyz': None}
    with mol_output:
        clear_output()
        print("Please enter XYZ coordinates or SMILES.")


def handle_submit(button):
    with output_area:
        clear_output()
        
        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        control_content = control_widget.value
        raw_input = coords_widget.value.strip()
        
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return
        
        control_errors = validate_control_text(control_content)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return
        
        # Input verarbeiten
        input_content, input_type = clean_input_data(raw_input)
        
        # Bei SMILES: Verwende konvertierte XYZ-Koordinaten
        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
            input_type = 'xyz (from SMILES)'
        
        if not input_content:
            print("Error: No valid input found!")
            return
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        time_limit = JOB_TIME_LIMITS.get(job_type, '48:00:00')
        
        try:
            job_dir.mkdir(parents=True, exist_ok=True)
            
            # Write SMILES to CONTROL if available
            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace(
                    'SMILES=',
                    f"SMILES={_converted_xyz_cache['smiles']}"
                )
            
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)
            
            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)
            
            pal, maxcore = parse_resource_settings(control_content)
            total_mem = pal * maxcore if pal and maxcore else 240000
            
            result = submit_job(
                job_dir=job_dir,
                mode='delfin',
                job_name=safe_job_name,
                time_limit=time_limit,
                ntasks=pal or 40,
                mem_mb=total_mem
            )
            
            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print(f"Job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Job Type: {job_type}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Directory: {job_dir}")
                print(f"")
                print(f"Check status: squeue -u $USER")
                
                reset_form()
            else:
                print(f"Error submitting job:")
                print(result.stderr)
        
        except Exception as e:
            print(f"Error creating job: {str(e)}")


def handle_validate_control(button):
    with validate_output:
        clear_output()
        control_content = control_widget.value
        control_errors = validate_control_text(control_content)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
        else:
            print("CONTROL.txt looks valid.")


def handle_only_goat_submit(button):
    with only_goat_output:
        clear_output()

        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        raw_input = coords_widget.value.strip()
        charge_value = only_goat_charge.value
        solvent_value = only_goat_solvent.value.strip()

        if not job_name:
            print("Error: Job name cannot be empty!")
            return

        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return

        if solvent_value == '':
            print("Error: Solvent cannot be empty!")
            return

        input_content, input_type = clean_input_data(raw_input)
        
        # Bei SMILES: Verwende konvertierte XYZ-Koordinaten
        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
            input_type = 'xyz (from SMILES)'

        if not input_content:
            print("Error: No valid input found!")
            return

        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return

        if not ONLY_GOAT_TEMPLATE_PATH.exists():
            print(f"Error: Only GOAT template not found: {ONLY_GOAT_TEMPLATE_PATH}")
            return

        job_dir = BASE_DIR / safe_job_name
        time_limit = JOB_TIME_LIMITS.get(job_type, '48:00:00')

        try:
            job_dir.mkdir(parents=True, exist_ok=True)

            control_template = ONLY_GOAT_TEMPLATE_PATH.read_text()
            control_content = (control_template
                .replace('[CHARGE]', str(charge_value))
                .replace('[SOLVENT]', solvent_value)
            )
            
            # Write SMILES to CONTROL if available
            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace(
                    'SMILES=',
                    f"SMILES={_converted_xyz_cache['smiles']}"
                )
            
            control_errors = validate_control_text(control_content)
            if control_errors:
                print("CONTROL.txt validation failed:")
                for err in control_errors:
                    print(f"- {err}")
                return
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)

            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)

            pal, maxcore = parse_resource_settings(control_content)
            total_mem = pal * maxcore if pal and maxcore else 240000

            result = submit_job(
                job_dir=job_dir,
                mode='delfin',
                job_name=safe_job_name,
                time_limit=time_limit,
                ntasks=pal or 40,
                mem_mb=total_mem
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
                print("Only GOAT job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Job Type: {job_type}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Charge: {charge_value}")
                print(f"Solvent: {solvent_value}")
                print(f"Directory: {job_dir}")
                print(f"")
                print(f"Check status: squeue -u $USER")

                reset_form()
            else:
                print("Error submitting job:")
                print(result.stderr)

        except Exception as e:
            print(f"Error creating job: {str(e)}")


only_goat_submit_button.on_click(handle_only_goat_submit)
validate_button.on_click(handle_validate_control)
submit_button.on_click(handle_submit)

spacer = widgets.Label(value="", layout=widgets.Layout(height='20px'))

ui_left = widgets.VBox([
    job_name_widget,
    spacer,
    job_type_widget,
    spacer,
    coords_help,
    coords_widget,
    button_spacer,
    convert_smiles_button,
    spacer,
    smiles_batch_help,
    smiles_batch_widget,
    button_spacer,
    submit_smiles_list_button,
    smiles_batch_output,
    spacer,
    control_help,
    control_widget,
    button_spacer,
    validate_button,
    submit_button,
    output_area,
    validate_output
], layout=widgets.Layout(width='55%', padding='10px'))

ui_right = widgets.VBox([
    widgets.Label("Molecule visualization:"),
    mol_output,
    xyz_copy_row,
    spacer,
    only_goat_label,
    widgets.HBox([
        only_goat_charge,
        only_goat_solvent,
        only_goat_submit_button
    ], layout=widgets.Layout(width='600px')),
    only_goat_output
], layout=widgets.Layout(width='45%', padding='10px'))

full_ui = widgets.HBox([ui_left, ui_right])
tab1_content = full_ui

# === DELFIN RECALC MODULES ===
import re
import subprocess
from IPython.display import display, clear_output
import ipywidgets as widgets

recalc_title = widgets.HTML("<h3>DELFIN RECALC MODULES</h3>")
recalc_help = widgets.Label("Select a job folder from calc, edit CONTROL.txt, then submit a recalc job.")

recalc_folder_dropdown = widgets.Dropdown(
    options=[],
    description='Job Folder:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': 'initial'}
)

recalc_time_limit = widgets.ToggleButtons(
    options=['12h', '24h', '36h', '48h', '60h', '72h'],
    value='24h',
    description='Time Limit:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='550px')
)

recalc_control_widget = widgets.Textarea(
    value='',
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='400px'),
    style={'description_width': 'initial'}
)

recalc_spacer = widgets.Label(value='', layout=widgets.Layout(height='10px'))

recalc_button = widgets.Button(
    description='SUBMIT RECALC',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)
recalc_output = widgets.Output()


def refresh_recalc_folders():
    if not BASE_DIR.exists():
        recalc_folder_dropdown.options = []
        recalc_folder_dropdown.value = None
        return

    folders = sorted([p.name for p in BASE_DIR.iterdir() if p.is_dir()])
    recalc_folder_dropdown.options = folders
    recalc_folder_dropdown.value = None


def load_recalc_control(change=None):
    with recalc_output:
        clear_output()

        folder = recalc_folder_dropdown.value
        if not folder:
            recalc_control_widget.value = ''
            return

        control_path = BASE_DIR / folder / 'CONTROL.txt'
        if control_path.exists():
            recalc_control_widget.value = control_path.read_text()
        else:
            recalc_control_widget.value = ''
            print(f"CONTROL.txt not found in {control_path.parent}")


def handle_recalc(button):
    with recalc_output:
        clear_output()

        folder = recalc_folder_dropdown.value
        if not folder:
            print("Error: Please select a job folder.")
            return

        job_dir = BASE_DIR / folder
        if not job_dir.exists():
            print(f"Error: Job folder does not exist: {job_dir}")
            return

        # Write updated CONTROL.txt
        control_errors = validate_control_text(recalc_control_widget.value)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return

        control_path = job_dir / 'CONTROL.txt'
        control_path.write_text(recalc_control_widget.value)

        pal, maxcore = parse_resource_settings(recalc_control_widget.value)
        if pal is None or maxcore is None:
            print('Error: PAL/maxcore not found in CONTROL.txt')
            return
        total_mem = pal * maxcore

        time_limit = JOB_TIME_LIMITS.get(recalc_time_limit.value, '24:00:00')

        result = submit_job(
            job_dir=job_dir,
            mode='delfin-recalc',
            job_name=job_dir.name,
            time_limit=time_limit,
            ntasks=pal,
            mem_mb=total_mem
        )

        if result.returncode == 0:
            # sbatch output typically: "Submitted batch job <id>"
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("Recalc job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Directory: {job_dir}")
        else:
            print("Error submitting recalc job:")
            print(result.stderr or result.stdout)


recalc_folder_dropdown.observe(load_recalc_control, names='value')
refresh_recalc_folders()

recalc_section = widgets.VBox([
    recalc_title,
    recalc_help,
    recalc_folder_dropdown,
    recalc_time_limit,
    recalc_control_widget,
    recalc_spacer,
    recalc_button,
    recalc_output
], layout=widgets.Layout(width='600px', padding='10px'))

recalc_button.on_click(handle_recalc)

tab2_content = recalc_section

# === Interactive Job Status ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
import json
import json
import subprocess
import os

job_status_title = widgets.HTML("<h3>Job Status</h3>")
job_status_help = widgets.HTML("<p>Select a job from the dropdown and click CANCEL JOB to cancel it. Click REFRESH to update the list.</p>")

# Speichere Job-Daten
_job_data = []

# HTML-Tabelle für Job-Anzeige
job_table_html = widgets.HTML(value="<i>Loading...</i>")

# Startzeit-Anzeige für wartende Jobs
job_start_html = widgets.HTML(value="")

# Dropdown für Job-Auswahl
job_dropdown = widgets.Dropdown(
    options=[],
    description='Select Job:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '80px'}
)

job_status_output = widgets.Output()

def refresh_job_list(button=None):
    """Refresh the job list from squeue."""
    global _job_data
    
    with job_status_output:
        clear_output()
    
    try:
        # Formatierte squeue Ausgabe für bessere Spalten
        result = subprocess.run(
            ['squeue', '-u', os.environ.get('USER', ''), 
             '-o', '%.12i %.12P %.35j %.10u %.3t %.12M %.6D %R'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        # Startzeit für wartende Jobs abrufen
        start_result = subprocess.run(
            ['squeue', '-u', os.environ.get('USER', ''), '--start',
             '-o', '%.12i %.35j %.20S'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if result.returncode == 0:
            lines = result.stdout.strip().split('\n')
            
            if len(lines) < 2:
                job_table_html.value = "<p><i>No jobs in queue.</i></p>"
                job_start_html.value = ""
                job_dropdown.options = []
                _job_data = []
                return
            
            header = lines[0]
            job_lines = lines[1:]
            
            # Parse jobs
            _job_data = []
            dropdown_options = []
            
            for line in job_lines:
                if not line.strip():
                    continue
                parts = line.split()
                if len(parts) >= 1:
                    job_id = parts[0].strip()
                    job_name = parts[2].strip() if len(parts) > 2 else "unknown"
                    _job_data.append({'id': job_id, 'line': line, 'name': job_name})
                    dropdown_options.append((f"{job_id} - {job_name}", job_id))
            
            # Build HTML table
            table_html = """
            <style>
                .job-table { font-family: monospace; font-size: 12px; border-collapse: collapse; width: 100%; }
                .job-table th, .job-table td { padding: 6px 10px; text-align: left; border-bottom: 1px solid #ddd; }
                .job-table th { background-color: #2196F3; color: white; }
                .job-table tr:hover { background-color: #f5f5f5; }
            </style>
            <table class="job-table">
                <tr>
                    <th>JOBID</th>
                    <th>PARTITION</th>
                    <th>NAME</th>
                    <th>USER</th>
                    <th>ST</th>
                    <th>TIME</th>
                    <th>NODES</th>
                    <th>NODELIST/REASON</th>
                </tr>
            """
            
            for job in _job_data:
                parts = job['line'].split(None, 7)  # Max 8 parts
                while len(parts) < 8:
                    parts.append('')
                table_html += f"""
                <tr>
                    <td><b>{parts[0]}</b></td>
                    <td>{parts[1]}</td>
                    <td>{parts[2]}</td>
                    <td>{parts[3]}</td>
                    <td>{parts[4]}</td>
                    <td>{parts[5]}</td>
                    <td>{parts[6]}</td>
                    <td>{parts[7]}</td>
                </tr>
                """
            
            table_html += "</table>"
            job_table_html.value = table_html
            
            # Parse and display start times for pending jobs
            if start_result.returncode == 0:
                start_lines = start_result.stdout.strip().split('\n')
                if len(start_lines) > 1:
                    pending_jobs = []
                    for line in start_lines[1:]:
                        if line.strip() and 'N/A' not in line:
                            parts = line.split(None, 2)
                            if len(parts) >= 3:
                                pending_jobs.append({
                                    'id': parts[0].strip(),
                                    'name': parts[1].strip(),
                                    'start': parts[2].strip()
                                })
                    
                    if pending_jobs:
                        start_html = """
                        <div style="margin-top: 10px; padding: 10px; background-color: #fff3cd; border: 1px solid #ffc107; border-radius: 4px;">
                            <b>Estimated Start Times (pending jobs):</b><br>
                            <table style="font-family: monospace; font-size: 12px; margin-top: 5px;">
                        """
                        for pj in pending_jobs:
                            start_html += f"<tr><td><b>{pj['id']}</b></td><td style='padding-left:15px;'>{pj['name']}</td><td style='padding-left:15px;'>{pj['start']}</td></tr>"
                        start_html += "</table></div>"
                        job_start_html.value = start_html
                    else:
                        job_start_html.value = ""
                else:
                    job_start_html.value = ""
            else:
                job_start_html.value = ""
            
            # Update dropdown
            if dropdown_options:
                job_dropdown.options = dropdown_options
                job_dropdown.value = None
            else:
                job_dropdown.options = []
            
            with job_status_output:
                clear_output()
                print(f"{len(_job_data)} job(s) found")
        else:
            with job_status_output:
                clear_output()
                print(f"Error: {result.stderr}")
    except Exception as e:
        with job_status_output:
            clear_output()
            print(f"Error fetching jobs: {e}")

def cancel_selected_job(button):
    """Cancel the currently selected job."""
    with job_status_output:
        clear_output()
        
        if not job_dropdown.value:
            print("No job selected.")
            return
        
        job_id = job_dropdown.value
        
        result = subprocess.run(
            ['scancel', str(job_id)],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print(f"Job {job_id} cancelled successfully.")
            # Refresh the list
            refresh_job_list()
        else:
            print(f"Error cancelling job {job_id}:")
            print(result.stderr or result.stdout)

# Buttons
refresh_button = widgets.Button(
    description='REFRESH',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

cancel_button = widgets.Button(
    description='CANCEL JOB',
    button_style='danger',
    layout=widgets.Layout(width='150px')
)

refresh_button.on_click(refresh_job_list)
cancel_button.on_click(cancel_selected_job)

# Initial load
refresh_job_list()

# Layout
button_row = widgets.HBox([
    job_dropdown,
    cancel_button,
    refresh_button
], layout=widgets.Layout(margin='10px 0', align_items='center'))

job_status_section = widgets.VBox([
    job_status_title,
    job_status_help,
    job_table_html,
    job_start_html,
    button_row,
    job_status_output
], layout=widgets.Layout(width='600px', padding='10px', border='1px solid #2196F3'))

tab3_content = job_status_section

# === ORCA Input Builder ===
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import re
from pathlib import Path
from delfin.common.control_validator import ORCA_FUNCTIONALS, ORCA_BASIS_SETS, DISP_CORR_VALUES, _RI_JKX_KEYWORDS

# Ensure BASE_DIR is available even if the main dashboard cell wasn't run
try:
    BASE_DIR
except NameError:
    if 'ROOT_DIR' in globals():
        BASE_DIR = ROOT_DIR / 'calc'
    else:
        BASE_DIR = Path.cwd() / 'calc'



orca_method_options = sorted(ORCA_FUNCTIONALS)
orca_basis_options = sorted(ORCA_BASIS_SETS)
orca_dispersion_options = ['None'] + sorted(v for v in DISP_CORR_VALUES if v)
orca_ri_options = ['None'] + sorted(_RI_JKX_KEYWORDS)
orca_aux_basis_options = ['None', 'def2/J', 'def2/JK']

orca_builder_title = widgets.HTML("<h3>ORCA Input Builder</h3>")
orca_builder_help = widgets.Label("Build ORCA input files and submit standalone ORCA calculations.")

# === ORCA Builder Widgets ===
orca_job_name = widgets.Text(
    value='',
    placeholder='e.g. water_opt',
    description='Job Name:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '120px'}
)

orca_coords = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates (with or without header):\n\n6\nComment line\nC  0.0  0.0  0.0\n...\n\nor just:\nC  0.0  0.0  0.0\n...:\nO   0.000000   0.000000   0.117300\nH   0.000000   0.756950  -0.469200\nH   0.000000  -0.756950  -0.469200',
    description='Coordinates:',
    layout=widgets.Layout(width='650px', height='400px'),
    style={'description_width': '120px'}
)

orca_charge = widgets.IntText(
    value=0,
    description='Charge:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_multiplicity = widgets.IntText(
    value=1,
    description='Multiplicity:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_method = widgets.Dropdown(
    options=orca_method_options,
    value='PBE0',
    description='Method:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_job_type = widgets.Dropdown(
    options=['SP', 'OPT', 'FREQ', 'OPT FREQ'],
    value='OPT',
    description='Job Type:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_basis = widgets.Dropdown(
    options=orca_basis_options,
    value='def2-SVP',
    description='Basis Set:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_dispersion = widgets.Dropdown(
    options=orca_dispersion_options,
    value='D4',
    description='Dispersion:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_ri = widgets.Dropdown(
    options=orca_ri_options,
    value='RIJCOSX',
    description='RI Approx:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_aux_basis = widgets.Dropdown(
    options=orca_aux_basis_options,
    value='def2/J',
    description='Aux Basis:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_autoaux = widgets.Checkbox(
    value=False,
    description='AutoAux',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

# === Solvation (CPCM/SMD) ===
orca_solvation_type = widgets.Dropdown(
    options=['None', 'CPCM', 'SMD'],
    value='None',
    description='Solvation Type:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_solvent = widgets.Dropdown(
    options=['water', 'acetonitrile', 'dmso', 'dmf', 'methanol', 'ethanol', 'thf', 'dichloromethane', 'chloroform', 'toluene', 'hexane'],
    value='water',
    description='Solvent:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

# === Print options ===
orca_print_mos = widgets.Checkbox(
    value=False,
    description='Print MOs',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_print_basis = widgets.Checkbox(
    value=False,
    description='Print Basis',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)



orca_additional = widgets.Text(
    value='',
    placeholder='e.g. FinalGrid6 NormalPrint',
    description='Additional:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '120px'}
)

orca_pal = widgets.IntText(
    value=40,
    description='PAL (cores):',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_maxcore = widgets.IntText(
    value=6000,
    description='MaxCore (MB):',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_slurm_time = widgets.Text(
    value='12:00:00',
    placeholder='e.g. 02:00:00',
    description='Time Limit:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)



# === File Upload (Drag & Drop) ===
orca_file_upload = widgets.FileUpload(
    accept='',
    multiple=True,
    description='Extra Files:',
    layout=widgets.Layout(width='500px', height='80px'),
    style={'description_width': '120px'}
)

orca_uploaded_files_label = widgets.HTML(
    value='<i>Drag & drop files here (e.g. .gbw, .xyz, .hess)</i>',
    layout=widgets.Layout(width='500px', height='80px')
)

# Preview area
orca_preview = widgets.Textarea(
    value='',
    description='INP Preview:',
    layout=widgets.Layout(width='600px', height='550px'),
    style={'description_width': '120px'},
    disabled=False
)

# Buttons
orca_generate_btn = widgets.Button(
    description='GENERATE INP',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

orca_save_btn = widgets.Button(
    description='SAVE INP + SH',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

orca_submit_btn = widgets.Button(
    description='SUBMIT ORCA JOB',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

orca_output = widgets.Output()

# Cache for extra uploaded files across multiple selections
_orca_extra_files = {}

orca_mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #4CAF50',
    width='600px',
    height='450px',
    overflow='hidden'
))


def reset_orca_builder():
    """Reset all ORCA Builder fields to their default values."""
    orca_job_name.value = ''
    orca_coords.value = ''
    orca_charge.value = 0
    orca_multiplicity.value = 1
    orca_method.value = 'PBE0'
    orca_job_type.value = 'OPT'
    orca_basis.value = 'def2-SVP'
    orca_dispersion.value = 'D4'
    orca_ri.value = 'RIJCOSX'
    orca_aux_basis.value = 'def2/J'
    orca_solvation_type.value = 'None'
    orca_solvent.value = 'water'
    orca_print_mos.value = False
    orca_print_basis.value = False
    orca_additional.value = ''
    orca_pal.value = 40
    orca_maxcore.value = 6000
    orca_slurm_time.value = '12:00:00'
    orca_file_upload.value = ()  # Clear uploaded files
    orca_uploaded_files_label.value = ''
    _orca_extra_files.clear()
    update_orca_preview()



def strip_xyz_header(text):
    """Remove XYZ header (atom count + comment line) if present."""
    text = text.strip()
    if not text:
        return text
    
    lines = text.split('\n')
    if len(lines) < 2:
        return text
    
    first_line = lines[0].strip()
    try:
        int(first_line)  # If first line is a number, it's an atom count
        return '\n'.join(lines[2:]).strip()  # Skip first 2 lines
    except ValueError:
        return text  # No header, return as-is


def update_orca_molecule_view(change=None):
    """Update molecule visualization for ORCA Builder."""
    with orca_mol_output:
        clear_output()
        raw_input = orca_coords.value.strip()
        
        if not raw_input:
            print("Paste XYZ coordinates to see 3D preview.")
            return
        
        # Strip header if present
        coords = strip_xyz_header(raw_input)
        
        if not coords:
            print("No valid coordinates.")
            return
        
        try:
            lines = [l for l in coords.split('\n') if l.strip()]
            num_atoms = len(lines)
            xyz_data = f"{num_atoms}\nORCA Builder Preview\n{coords}"
            mol_view = py3Dmol.view(width=590, height=440)
            mol_view.addModel(xyz_data, "xyz")
            mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
            mol_view.zoomTo()
            mol_view.show()
        except Exception as e:
            print(f"Could not visualize: {e}")


def generate_orca_input():
    """Generate ORCA input file content from widget values."""
    keywords = []
    
    # Method
    keywords.append(orca_method.value)
    
    # Job type
    keywords.append(orca_job_type.value)
    
    # Basis set
    keywords.append(orca_basis.value)
    
    # Dispersion
    if orca_dispersion.value != 'None':
        keywords.append(orca_dispersion.value)
    
    # RI approximation
    if orca_ri.value != 'None':
        keywords.append(orca_ri.value)
    
    # Auxiliary basis
    if orca_ri.value != 'None':
        keywords.append(orca_aux_basis.value)
        if orca_autoaux.value:
            keywords.append('AutoAux')
    
    # Solvation - simple syntax: TYPE(solvent)
    if orca_solvation_type.value != 'None':
        keywords.append(f"{orca_solvation_type.value}({orca_solvent.value})")
    
    # Output block
    output_lines = []
    if orca_print_mos.value:
        output_lines.append("  print[p_mos] 1")
    if orca_print_basis.value:
        output_lines.append("  print[p_basis] 2")
    output_block = ""
    if output_lines:
        output_block = "%output\n" + "\n".join(output_lines) + "\nend"
    
    # Additional keywords
    if orca_additional.value.strip():
        keywords.append(orca_additional.value.strip())
    
    keyword_line = '! ' + ' '.join(keywords)
    
    # PAL block
    pal_block = f"""%pal
  nprocs {orca_pal.value}
end"""
    
    # MaxCore
    maxcore_line = f"%maxcore {orca_maxcore.value}"
    
    # Coordinates block
    coords = strip_xyz_header(orca_coords.value)
    coord_block = f"""* xyz {orca_charge.value} {orca_multiplicity.value}
{coords}
*"""
    
    # Assemble full input
    inp_content = f"""{keyword_line}

{pal_block}

{maxcore_line}
"""
    if output_block:
        inp_content += f"\n{output_block}\n"
    inp_content += f"\n{coord_block}\n"
    return inp_content

def parse_inp_resources(inp_text):
    """Parse PAL (nprocs) and maxcore from ORCA input text."""
    pal = None
    maxcore = None
    if not inp_text:
        return pal, maxcore
    pal_match = re.search(r'(?im)^\s*nprocs\s+(\d+)', inp_text)
    if pal_match:
        pal = int(pal_match.group(1))
    maxcore_match = re.search(r'(?im)^\s*%maxcore\s+(\d+)', inp_text)
    if maxcore_match:
        maxcore = int(maxcore_match.group(1))
    return pal, maxcore


def save_uploaded_files(job_dir):
    """Save uploaded files to job directory."""
    saved_files = []
    files_to_save = _orca_extra_files if _orca_extra_files else {}
    # Fallback to current widget selection if cache is empty
    if not files_to_save and orca_file_upload.value:
        for uploaded_file in orca_file_upload.value:
            filename = uploaded_file['name'] if isinstance(uploaded_file, dict) else uploaded_file.name
            content = uploaded_file['content'] if isinstance(uploaded_file, dict) else uploaded_file.content
            files_to_save[filename] = content
    for filename, content in files_to_save.items():
        file_path = job_dir / filename
        file_path.write_bytes(content)
        saved_files.append(filename)
    return saved_files


def update_uploaded_files_label(change=None):
    """Update the label showing uploaded files."""
    if orca_file_upload.value:
        for f in orca_file_upload.value:
            name = f['name'] if isinstance(f, dict) else f.name
            content = f['content'] if isinstance(f, dict) else f.content
            _orca_extra_files[name] = content
    if _orca_extra_files:
        filenames = list(_orca_extra_files.keys())
        orca_uploaded_files_label.value = f"<b>Files to upload:</b> {', '.join(filenames)}"
    else:
        orca_uploaded_files_label.value = '<i>Drag & drop files here (e.g. .gbw, .xyz, .hess)</i>'


orca_file_upload.observe(update_uploaded_files_label, names='value')


def update_orca_preview(change=None):
    """Update the ORCA input preview."""
    orca_preview.value = generate_orca_input()


def handle_orca_generate(button):
    """Handle GENERATE INP button click."""
    update_orca_preview()
    with orca_output:
        clear_output()
        print("ORCA input generated. You can edit the preview if needed.")


def handle_orca_save(button):
    """Handle SAVE INP + SH button click."""
    with orca_output:
        clear_output()
        
        job_name = orca_job_name.value.strip()
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        job_dir.mkdir(parents=True, exist_ok=True)
        
        preview_content = orca_preview.value.strip()
        if preview_content:
            inp_content = preview_content
        else:
            coords = strip_xyz_header(orca_coords.value)
            if not coords:
                print("Error: Coordinates or INP preview cannot be empty!")
                return
            inp_content = generate_orca_input()
        
        # Save .inp file
        inp_path = job_dir / f"{safe_job_name}.inp"
        inp_path.write_text(inp_content)
        
        # Save uploaded files
        saved_files = save_uploaded_files(job_dir)
        
        # Parse resources from input
        pal_used, maxcore_used = parse_inp_resources(inp_content)
        if pal_used is None:
            pal_used = orca_pal.value
        if maxcore_used is None:
            maxcore_used = orca_maxcore.value
        total_mem = pal_used * maxcore_used
        
        # Submit job using central submit function
        result = submit_job(
            job_dir=job_dir,
            mode='orca',
            job_name=safe_job_name,
            inp_file=f'{safe_job_name}.inp',
            time_limit=orca_slurm_time.value,
            ntasks=pal_used,
            mem_mb=total_mem
        )
        
        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("ORCA job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Job Name: {safe_job_name}")
            print(f"Directory: {job_dir}")
            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            print(f"PAL: {pal_used}, MaxCore: {maxcore_used} MB")
            print(f"Time Limit: {orca_slurm_time.value}")
            if orca_solvation_type.value != 'None':
                print(f"Solvation: {orca_solvation_type.value}({orca_solvent.value})")
            if saved_files:
                print(f"Extra files: {', '.join(saved_files)}")
            print()
            print("Check status: squeue -u $USER")
            # Reset form for next job
            reset_orca_builder()
        else:
            print("Error submitting job:")
            print(result.stderr or result.stdout)


def handle_orca_submit(button):
    """Handle SUBMIT ORCA JOB button click."""
    with orca_output:
        clear_output()
        
        job_name = orca_job_name.value.strip()
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        preview_content = orca_preview.value.strip()
        if preview_content:
            inp_content = preview_content
        else:
            coords = strip_xyz_header(orca_coords.value)
            if not coords:
                print("Error: Coordinates or INP preview cannot be empty!")
                return
            inp_content = generate_orca_input()
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        job_dir.mkdir(parents=True, exist_ok=True)
        
        # Save .inp file
        inp_path = job_dir / f"{safe_job_name}.inp"
        inp_path.write_text(inp_content)
        
        # Save uploaded files
        saved_files = save_uploaded_files(job_dir)
        
        # Parse resources from input
        pal_used, maxcore_used = parse_inp_resources(inp_content)
        if pal_used is None:
            pal_used = orca_pal.value
        if maxcore_used is None:
            maxcore_used = orca_maxcore.value
        total_mem = pal_used * maxcore_used
        
        # Submit job using central submit function
        result = submit_job(
            job_dir=job_dir,
            mode='orca',
            job_name=safe_job_name,
            inp_file=f'{safe_job_name}.inp',
            time_limit=orca_slurm_time.value,
            ntasks=pal_used,
            mem_mb=total_mem
        )
        
        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("ORCA job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Job Name: {safe_job_name}")
            print(f"Directory: {job_dir}")
            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            print(f"PAL: {pal_used}, MaxCore: {maxcore_used} MB")
            print(f"Time Limit: {orca_slurm_time.value}")
            if orca_solvation_type.value != 'None':
                print(f"Solvation: {orca_solvation_type.value}({orca_solvent.value})")
            if saved_files:
                print(f"Extra files: {', '.join(saved_files)}")
            print()
            print("Check status: squeue -u $USER")
            # Reset form for next job
            reset_orca_builder()
        else:
            print("Error submitting job:")
            print(result.stderr or result.stdout)


# Connect buttons
orca_generate_btn.on_click(handle_orca_generate)
orca_save_btn.on_click(handle_orca_save)
orca_submit_btn.on_click(handle_orca_submit)

# Connect widget changes to preview update
for w in [orca_method, orca_job_type, orca_basis, orca_dispersion, orca_ri, 
          orca_aux_basis, orca_charge, orca_multiplicity, orca_pal, orca_maxcore,
          orca_coords, orca_additional, orca_solvation_type, orca_solvent,
          orca_print_mos, orca_print_basis, orca_slurm_time]:
    w.observe(update_orca_preview, names='value')

# Initial preview

# Connect coords to molecule viewer
orca_coords.observe(update_orca_molecule_view, names='value')

# Initial molecule view
update_orca_molecule_view()

update_orca_preview()

# Layout
orca_row1 = widgets.HBox([orca_job_name])
orca_row2 = widgets.HBox([orca_coords])
orca_row3 = widgets.HBox([orca_charge, orca_multiplicity])
orca_row4 = widgets.HBox([orca_method, orca_job_type])
orca_row5 = widgets.HBox([orca_basis, orca_dispersion])
orca_row6 = widgets.HBox([orca_ri, orca_aux_basis])
orca_row6b = widgets.HBox([orca_autoaux])
orca_row_solv = widgets.HBox([orca_solvation_type, orca_solvent])
orca_row_opts = widgets.HBox([orca_print_mos, orca_print_basis])
orca_row7 = widgets.HBox([orca_additional])
orca_row8 = widgets.HBox([orca_pal, orca_maxcore])
orca_row9 = widgets.HBox([orca_slurm_time])
orca_row_upload = widgets.VBox(
    [orca_file_upload, orca_uploaded_files_label],
    layout=widgets.Layout(margin='0 0 0 120px')
)
orca_row_buttons = widgets.HBox(
    [orca_generate_btn, orca_save_btn, orca_submit_btn],
    layout=widgets.Layout(margin='0 0 0 120px')
)

orca_left = widgets.VBox([
    orca_row1,
    orca_row2,
    orca_row3,
    orca_row4,
    orca_row5,
    orca_row6,
    orca_row6b,
    orca_row_solv,
    orca_row_opts,
    orca_row7,
    orca_row8,
    orca_row9,
    orca_row_upload,
    orca_row_buttons,
    orca_output
], layout=widgets.Layout(width='50%', padding='10px'))

orca_right = widgets.VBox([
    widgets.Label("Live ORCA Input Preview (editable):"),
    orca_preview,
    widgets.HTML('<b>Molecule Preview:</b>', layout=widgets.Layout(margin='10px 0 0 0')),
    orca_mol_output
], layout=widgets.Layout(width='50%', padding='10px'))

orca_builder_ui = widgets.HBox([orca_left, orca_right])

orca_builder_section = widgets.VBox([
    orca_builder_title,
    orca_builder_help,
    orca_builder_ui
], layout=widgets.Layout(width='600px', padding='10px', border='1px solid #4CAF50'))

tab4_content = orca_builder_section


# === Layout (Dashboard6 style) ===
spacer = widgets.Label(value='', layout=widgets.Layout(height='10px'))
spacer_large = widgets.Label(value='', layout=widgets.Layout(height='20px'))

submit_left = widgets.VBox([
    job_name_widget, spacer, job_type_widget, spacer_large,
    widgets.HTML('<b>Input (XYZ or SMILES):</b>'), coords_widget, spacer, convert_smiles_button,
    spacer_large, widgets.HTML('<b>Batch SMILES:</b>'), smiles_batch_widget, spacer, submit_smiles_list_button, smiles_batch_output,
    spacer_large, widgets.HTML('<b>CONTROL.txt:</b>'), control_widget, spacer,
    widgets.HBox([validate_button, submit_button]), output_area, validate_output
], layout=widgets.Layout(width='50%', padding='10px'))

submit_right = widgets.VBox([
    widgets.HTML('<b>Molecule Preview:</b>'), mol_output,
    spacer_large, widgets.HTML('<b>Only GOAT:</b>'),
    widgets.HBox([only_goat_charge, only_goat_solvent, only_goat_submit_button]),
    only_goat_output
], layout=widgets.Layout(width='50%', padding='10px'))

tab1_content = widgets.HBox([submit_left, submit_right])

recalc_refresh_btn = widgets.Button(description='REFRESH FOLDERS', button_style='info', layout=widgets.Layout(width='150px'))
recalc_refresh_btn.on_click(lambda b: refresh_recalc_folders())

tab2_content = widgets.VBox([
    widgets.HTML('<h3>Recalc DELFIN Job</h3>'),
    widgets.HTML('<p>Select a job folder, edit CONTROL.txt, and resubmit.</p>'),
    widgets.HBox([recalc_folder_dropdown, recalc_refresh_btn]),
    recalc_time_limit,
    recalc_control_widget, spacer,
    recalc_button, recalc_output
], layout=widgets.Layout(padding='10px'))

tab3_content = widgets.VBox([
    widgets.HTML('<h3>Job Status</h3>'),
    widgets.HTML('<p>Select a job and click CANCEL to cancel it.</p>'),
    job_table_html, job_start_html,
    widgets.HBox([job_dropdown, cancel_button, refresh_button], layout=widgets.Layout(margin='10px 0')),
    job_status_output
], layout=widgets.Layout(padding='10px'))

orca_left = widgets.VBox([
    orca_row1,
    orca_row2,
    orca_row3,
    orca_row4,
    orca_row5,
    orca_row6,
    orca_row6b,
    orca_row_solv,
    orca_row_opts,
    orca_row7,
    orca_row8,
    orca_row9,
    orca_row_upload,
    orca_row_buttons,
    orca_output
], layout=widgets.Layout(width='50%', padding='10px'))

orca_right = widgets.VBox([
    widgets.HTML('<b>ORCA Input Preview (editable):</b>'),
    orca_preview,
    widgets.HTML('<b>Molecule Preview:</b>', layout=widgets.Layout(margin='10px 0 0 0')),
    orca_mol_output
], layout=widgets.Layout(width='50%', padding='10px'))

tab4_content = widgets.VBox([
    widgets.HTML('<h3>ORCA Input Builder</h3>'),
    widgets.HTML("<a href=\"https://orca-manual.mpi-muelheim.mpg.de/\" target=\"_blank\">ORCA User Manual</a>"),
    widgets.HBox([orca_left, orca_right])
], layout=widgets.Layout(padding='10px'))


# === Calculations Browser Tab ===
_calc_current_path = ''
_calc_file_content = ''
_calc_all_items = []  # All items in current directory
_calc_search_spans = []  # List of (start, end) positions
_calc_current_match = -1  # Current match index
_calc_selected_inp_path = None
_calc_selected_inp_base = ''
_calc_recalc_active = False
_calc_delete_current = False

calc_path_label = widgets.HTML(value='<b>📂 Path:</b> /', layout=widgets.Layout(width='100%', overflow_x='hidden'))

calc_back_btn = widgets.Button(description='⬆ Up', button_style='warning', layout=widgets.Layout(width='58px', height='26px'), disabled=True)
calc_home_btn = widgets.Button(description='🏠', button_style='info', layout=widgets.Layout(width='58px', height='26px'))
calc_refresh_btn = widgets.Button(description='🔄', layout=widgets.Layout(width='58px', height='26px'))
calc_delete_btn = widgets.Button(description='🗑 Delete', button_style='danger', layout=widgets.Layout(width='80px', height='26px'))
calc_delete_yes_btn = widgets.Button(description='Yes', button_style='danger', layout=widgets.Layout(width='60px', height='26px'))
calc_delete_no_btn = widgets.Button(description='No', layout=widgets.Layout(width='60px', height='26px'))
calc_delete_label = widgets.HTML('<b>Delete selected item?</b>')
calc_delete_confirm = widgets.HBox([
    calc_delete_label,
    calc_delete_yes_btn,
    calc_delete_no_btn
], layout=widgets.Layout(display='none', gap='6px', align_items='center'))
calc_delete_status = widgets.HTML(value='', layout=widgets.Layout(width='100%', overflow_x='hidden'))

# Folder search (filter files in directory)
calc_folder_search = widgets.Text(placeholder='Filter files...', continuous_update=True, layout=widgets.Layout(width='100%', height='26px', overflow_x='hidden', margin='0 0 8px 0'))

calc_file_list = widgets.Select(options=[], rows=22, layout=widgets.Layout(width='100%', height='520px'))

# Content area - HTML with highlights and scroll
calc_content_area = widgets.HTML(value='', layout=widgets.Layout(width='100%', display='block', overflow_x='hidden'))

calc_mol_label = widgets.HTML(
    "<div style='height:26px; line-height:26px; margin:0 0 8px 0;'><b>🔬 Molecule Preview:</b></div>"
)
calc_mol_viewer = widgets.Output(layout=widgets.Layout(width='520px', height='520px', border='2px solid #1976d2', overflow='hidden', padding='0', border_radius='0'))

# === Multi-frame XYZ trajectory support ===
_calc_xyz_frames = []  # List of (comment, xyz_content) tuples for each frame
_calc_xyz_current_frame = [0]
_calc_xyz_viewer_id = [None]  # Store viewer ID for frame switching  # Use list to avoid global issues

calc_xyz_frame_label = widgets.HTML(value='', layout=widgets.Layout(margin='4px 0'))
calc_xyz_prev_btn = widgets.Button(
    description='◀ Prev',
    button_style='info',
    layout=widgets.Layout(width='80px', height='32px')
)
calc_xyz_next_btn = widgets.Button(
    description='Next ▶',
    button_style='info',
    layout=widgets.Layout(width='80px', height='32px')
)
calc_xyz_frame_input = widgets.BoundedIntText(
    value=1, min=1, max=1, step=1,
    layout=widgets.Layout(width='60px', height='28px')
)
calc_xyz_frame_total = widgets.HTML(value='<b>/ 1</b>', layout=widgets.Layout(width='40px'))
calc_xyz_copy_btn = widgets.Button(
    description='📋 Copy Coordinates',
    button_style='success',
    layout=widgets.Layout(width='160px', height='32px')
)
calc_xyz_controls = widgets.HBox([
    widgets.HTML('<b>Frame:</b>'),
    calc_xyz_frame_input,
    calc_xyz_frame_total,
    calc_xyz_copy_btn
], layout=widgets.Layout(display='none', gap='6px', margin='6px 0', align_items='center'))

# Coord file copy button (for single Turbomole coord files, no frames)
calc_coord_copy_btn = widgets.Button(
    description='📋 Copy Coordinates',
    button_style='success',
    layout=widgets.Layout(width='160px', height='32px')
)
calc_coord_controls = widgets.HBox([
    calc_coord_copy_btn
], layout=widgets.Layout(display='none', gap='6px', margin='6px 0', align_items='center'))

def parse_xyz_frames(content):
    """Parse a multi-frame XYZ file into a list of (comment, xyz_block) tuples."""
    frames = []
    lines = content.strip().split('\n')
    i = 0
    while i < len(lines):
        # First line should be atom count
        try:
            n_atoms = int(lines[i].strip())
        except (ValueError, IndexError):
            break
        if i + 1 >= len(lines):
            break
        comment = lines[i + 1].strip()
        # Extract coordinate lines
        coord_lines = []
        for j in range(n_atoms):
            if i + 2 + j < len(lines):
                coord_lines.append(lines[i + 2 + j])
        if len(coord_lines) == n_atoms:
            xyz_block = '\n'.join(coord_lines)
            frames.append((comment, xyz_block, n_atoms))
        i += 2 + n_atoms
    return frames


def calc_update_xyz_viewer(initial_load=False):
    """Update the viewer for the current frame."""
    if not _calc_xyz_frames:
        return
    idx = _calc_xyz_current_frame[0]
    if idx < 0 or idx >= len(_calc_xyz_frames):
        return
    comment, xyz_block, n_atoms = _calc_xyz_frames[idx]
    
    # Update frame display (suppress observer temporarily)
    calc_xyz_frame_input.unobserve(calc_on_xyz_input_change, names='value')
    calc_xyz_frame_input.value = idx + 1
    calc_xyz_frame_input.max = len(_calc_xyz_frames)
    calc_xyz_frame_input.observe(calc_on_xyz_input_change, names='value')
    calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
    # Update label with comment
    calc_xyz_frame_label.value = f"{html.escape(comment[:100])}{'...' if len(comment) > 100 else ''}"
    
    if initial_load:
        # Build complete trajectory XYZ (all frames concatenated)
        full_xyz = ""
        for comm, block, natoms in _calc_xyz_frames:
            full_xyz += f"{natoms}\n{comm}\n{block}\n"
        
        # Create viewer with unique ID
        calc_mol_viewer.clear_output()
        with calc_mol_viewer:
            viewer_id = "calc_trj_viewer"
            html_content = f"""
            <div id="{viewer_id}" style="width:520px;height:520px;position:relative;"></div>
            <script>
            (function() {{
                var viewer = $3Dmol.createViewer("{viewer_id}", {{backgroundColor: "white"}});
                var xyz = `{full_xyz}`;
                viewer.addModelsAsFrames(xyz, "xyz");
                viewer.setStyle({{}}, {{stick: {{radius: 0.1}}, sphere: {{scale: 0.25}}}});
                viewer.zoomTo();
                viewer.setFrame(0);
                viewer.render();
                window.calc_trj_viewer = viewer;
            }})();
            </script>
            """
            display(HTML(html_content))
    else:
        # Just switch frame via JavaScript - don't clear output!
        with calc_mol_viewer:
            display(HTML(f"""
            <script>
            if(window.calc_trj_viewer) {{
                window.calc_trj_viewer.setFrame({idx});
                window.calc_trj_viewer.render();
            }}
            </script>
            """))


def calc_on_xyz_input_change(change):
    """Jump to entered frame number."""
    new_val = change['new']
    if _calc_xyz_frames and 1 <= new_val <= len(_calc_xyz_frames):
        _calc_xyz_current_frame[0] = new_val - 1
        calc_update_xyz_viewer(initial_load=False)


def calc_on_xyz_copy(button):
    """Copy current frame coordinates to clipboard via JavaScript."""
    global _calc_xyz_current_frame
    if not _calc_xyz_frames:
        return
    idx = _calc_xyz_current_frame[0]
    if idx < 0 or idx >= len(_calc_xyz_frames):
        return
    comment, xyz_block, n_atoms = _calc_xyz_frames[idx]
    # Build full XYZ content for clipboard
    xyz_content = f"{n_atoms}\n{comment}\n{xyz_block}"
    # Escape for JavaScript
    escaped = xyz_content.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code = f"navigator.clipboard.writeText('{escaped}').then(() => console.log('Copied frame {idx+1}')).catch(err => console.error('Copy failed:', err));"
    display(Javascript(js_code))


calc_xyz_frame_input.observe(calc_on_xyz_input_change, names='value')
calc_xyz_copy_btn.on_click(calc_on_xyz_copy)

def calc_on_coord_copy(button):
    """Copy coord file content to clipboard via JavaScript."""
    global _calc_file_content
    if not _calc_file_content:
        return
    # Escape for JavaScript
    escaped = _calc_file_content.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code = f"navigator.clipboard.writeText('{escaped}').then(() => console.log('Copied coord')).catch(err => console.error('Copy failed:', err));"
    display(Javascript(js_code))

calc_coord_copy_btn.on_click(calc_on_coord_copy)

calc_mol_container = widgets.VBox([calc_mol_label, calc_mol_viewer], layout=widgets.Layout(display='none', width='100%', align_items='flex-start'))

calc_file_info = widgets.HTML(value='', layout=widgets.Layout(width='100%', overflow_x='hidden'))
calc_content_label = widgets.HTML(value='<b>📄 File Content:</b>', layout=widgets.Layout(width='100%', overflow_x='hidden', margin='8px 0 0 0'))
calc_view_toggle = widgets.ToggleButton(description='Visualize', value=False, disabled=True, button_style='warning', layout=widgets.Layout(width='80px', height='26px'))

calc_recalc_btn = widgets.Button(description='Recalc', button_style='warning', layout=widgets.Layout(width='80px', height='26px'), disabled=True)
calc_submit_recalc_btn = widgets.Button(description='Submit Recalc', button_style='success', layout=widgets.Layout(width='130px', height='26px'), disabled=True)
calc_recalc_time = widgets.Text(value='24:00:00', description='Time limit', layout=widgets.Layout(width='200px', height='26px'))
calc_recalc_status = widgets.HTML(value='', layout=widgets.Layout(width='100%', overflow_x='hidden'))
calc_edit_area = widgets.Textarea(value='', layout=widgets.Layout(width='100%', height='520px', display='none'))

# Content navigation
calc_top_btn = widgets.Button(description='⬆ Top', layout=widgets.Layout(width='95px', height='26px'))
calc_bottom_btn = widgets.Button(description='⬇ End', layout=widgets.Layout(width='95px', height='26px'))

# Content search with navigation
calc_search_options = [
    'ABSORPTION SPECTRUM',
    'ERROR',
    'FINAL SINGLE POINT ENERGY',
    'Final Gibbs free energy',
    'HURRAY',
    'JOB NUMBER ',
    'LOEWDIN POPULATION ANALYSIS',
    'LOEWDIN REDUCED ORBITAL CHARGES',
    'MULLIKEN POPULATION ANALYSIS',
    'MULLIKEN REDUCED ORBITAL CHARGES',
    'ORBITAL ENERGIES',
    'SCF CONVERGED AFTER',
    'TD-DFT EXCITED STATES',
    'Total Enthalpy',
    'WARNING'
]
calc_search_input = widgets.Text(
    placeholder='Search in file...',
    continuous_update=False,
    layout=widgets.Layout(width='140px', height='26px')
)
calc_search_suggest = widgets.Dropdown(
    options=['(Select)'] + calc_search_options,
    value='(Select)',
    layout=widgets.Layout(width='200px', height='26px')
)
calc_search_btn = widgets.Button(description='🔍', layout=widgets.Layout(width='85px', height='26px'))
calc_prev_btn = widgets.Button(description='◀', layout=widgets.Layout(width='58px', height='26px'), disabled=True)
calc_next_btn = widgets.Button(description='▶', layout=widgets.Layout(width='58px', height='26px'), disabled=True)
calc_search_result = widgets.HTML(value='', layout=widgets.Layout(width='180px', min_width='180px'))

# === OCCUPIER Override Widgets ===
calc_options_dropdown = widgets.Dropdown(
    options=['(Options)'],
    value='(Options)',
    layout=widgets.Layout(width='150px', height='26px', visibility='hidden')
)
calc_override_input = widgets.Text(
    value='',
    placeholder='STAGE=INDEX',
    layout=widgets.Layout(width='140px', height='26px', visibility='hidden')
)
calc_override_time = widgets.Text(
    value='08:00:00',
    placeholder='HH:MM:SS',
    layout=widgets.Layout(width='80px', height='26px', visibility='hidden')
)
calc_override_btn = widgets.Button(
    description='Submit',
    button_style='success',
    layout=widgets.Layout(width='70px', height='26px', visibility='hidden')
)
calc_override_status = widgets.HTML(value='', layout=widgets.Layout(width='100%', visibility='hidden'))

# === OCCUPIER Override Handler ===
def calc_update_options_dropdown():
    """Update options dropdown based on selected file."""
    selected = calc_file_list.value
    sel_lower = selected.lower() if selected else ''
    if selected and 'OCCUPIER.txt' in selected:
        calc_options_dropdown.options = ['(Options)', 'Override']
        calc_options_dropdown.value = '(Options)'
        calc_options_dropdown.layout.visibility = 'visible'
    elif selected and ('CONTROL.txt' in selected or sel_lower.endswith('.inp')):
        calc_options_dropdown.options = ['(Options)', 'Recalc']
        calc_options_dropdown.value = '(Options)'
        calc_options_dropdown.layout.visibility = 'visible'
    else:
        calc_options_dropdown.options = ['(Options)']
        calc_options_dropdown.value = '(Options)'
        calc_options_dropdown.layout.visibility = 'hidden'
        calc_override_input.layout.visibility = 'hidden'
        calc_override_time.layout.visibility = 'hidden'
        calc_override_btn.layout.visibility = 'hidden'
        calc_override_status.layout.visibility = 'hidden'
        calc_override_status.value = ''
        calc_edit_area.layout.display = 'none'
        calc_content_area.layout.display = 'block'

def calc_on_options_change(change):
    """Show/hide override input when Override or Recalc is selected."""
    global _calc_file_content
    if change['new'] == 'Override':
        calc_override_input.layout.visibility = 'visible'
        calc_override_time.layout.visibility = 'visible'
        calc_override_btn.layout.visibility = 'visible'
        calc_override_status.layout.visibility = 'visible'
        calc_override_status.value = ''
        calc_edit_area.layout.display = 'none'
        calc_content_area.layout.display = 'block'
        # Pre-fill with stage name from filename
        selected = calc_file_list.value
        if selected:
            name = selected[2:].strip()
            # Extract stage name: e.g. "red_step_2_OCCUPIER.txt" -> "red_step_2"
            stage_name = name.replace('_OCCUPIER.txt', '').replace('OCCUPIER.txt', '')
            if stage_name:
                calc_override_input.value = f'{stage_name}='
            else:
                calc_override_input.value = ''
    elif change['new'] == 'Recalc':
        calc_override_input.layout.visibility = 'hidden'
        calc_override_time.layout.visibility = 'visible'
        calc_override_btn.layout.visibility = 'visible'
        calc_override_status.layout.visibility = 'visible'
        calc_override_status.value = ''
        # Show editable content
        calc_edit_area.value = _calc_file_content
        calc_edit_area.layout.display = 'block'
        calc_content_area.layout.display = 'none'
    else:
        calc_override_input.layout.visibility = 'hidden'
        calc_override_time.layout.visibility = 'hidden'
        calc_override_btn.layout.visibility = 'hidden'
        calc_override_status.layout.visibility = 'hidden'
        calc_override_status.value = ''
        calc_edit_area.layout.display = 'none'
        calc_content_area.layout.display = 'block'

def calc_find_workspace_root(current_path):
    """Find workspace root (directory containing CONTROL.txt) from current path."""
    if not current_path:
        return None
    
    # Start from current directory and search upwards
    path_parts = current_path.split('/')
    for i in range(len(path_parts), 0, -1):
        test_path = '/'.join(path_parts[:i])
        control_file = CALC_DIR / test_path / 'CONTROL.txt'
        if control_file.exists():
            return test_path
    return None

def calc_on_override_start(button):
    """Execute delfin --occupier-override, DELFIN recalc, or ORCA recalc."""
    global _calc_current_path, _calc_file_content
    
    is_recalc = calc_options_dropdown.value == 'Recalc'
    selected = calc_file_list.value
    selected_name = selected[2:].strip() if selected and len(selected) > 2 else ''
    is_inp_recalc = bool(is_recalc and selected_name.lower().endswith('.inp'))
    
    if not is_recalc:
        override_value = calc_override_input.value.strip()
        if not override_value or '=' not in override_value:
            calc_override_status.value = '<span style="color:#d32f2f;">Enter STAGE=INDEX (e.g., red_step_2=1)</span>'
            return

    # ORCA recalc for .inp selection
    if is_inp_recalc:
        inp_path = None
        if _calc_selected_inp_path and Path(_calc_selected_inp_path).exists():
            inp_path = Path(_calc_selected_inp_path)
        elif selected_name:
            inp_path = (CALC_DIR / _calc_current_path / selected_name) if _calc_current_path else (CALC_DIR / selected_name)
            if not inp_path.exists():
                inp_path = None
        if inp_path is None:
            calc_override_status.value = '<span style="color:#d32f2f;">No .inp file selected.</span>'
            return

        inp_content = calc_edit_area.value or inp_path.read_text()
        if not inp_content.strip():
            calc_override_status.value = '<span style="color:#d32f2f;">Input content is empty.</span>'
            return

        job_dir = inp_path.parent
        base_stem = inp_path.stem
        base_name = calc_get_recalc_base_name(base_stem)
        next_idx = calc_next_recalc_index(job_dir, base_name)
        job_name = f"{base_name}_recalc_{next_idx}"

        try:
            new_inp = job_dir / f"{job_name}.inp"
            new_inp.write_text(inp_content)

            slurm_time = calc_override_time.value.strip() or '24:00:00'

            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            total_mem = pal_used * maxcore_used

            result = submit_job(
                job_dir=job_dir,
                mode='orca',
                job_name=job_name,
                inp_file=f'{job_name}.inp',
                time_limit=slurm_time,
                ntasks=pal_used,
                mem_mb=total_mem
            )
        except Exception as exc:
            calc_override_status.value = f'<span style="color:#d32f2f;">Error: {html.escape(str(exc))}</span>'
            return

        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '(unknown)'
            calc_override_status.value = (
                f'<span style="color:#2e7d32;">Submitted {html.escape(job_name)} (ID: {html.escape(job_id)})</span>'
            )
        else:
            msg = result.stderr or result.stdout or 'Unknown sbatch error'
            calc_override_status.value = f'<span style="color:#d32f2f;">{html.escape(msg)}</span>'
        return
    
    # Find workspace root (where CONTROL.txt is located)
    if not _calc_current_path:
        calc_override_status.value = '<span style="color:#d32f2f;">No folder selected</span>'
        return
    
    workspace_root = calc_find_workspace_root(_calc_current_path)
    if not workspace_root:
        calc_override_status.value = '<span style="color:#d32f2f;">No CONTROL.txt found in parent directories</span>'
        return
    
    workspace_path = CALC_DIR / workspace_root
    
    # For Recalc: save edited CONTROL.txt first
    if is_recalc:
        try:
            control_path = workspace_path / 'CONTROL.txt'
            control_path.write_text(calc_edit_area.value)
            _calc_file_content = calc_edit_area.value
            calc_render_content(scroll_to='top')
        except Exception as e:
            calc_override_status.value = f'<span style="color:#d32f2f;">Error saving CONTROL.txt: {e}</span>'
            return
    
    calc_override_status.value = '<span style="color:#1976d2;">Submitting job...</span>'
    
    try:
        # Derive PAL/maxcore from CONTROL.txt first, fallback to .inp
        pal_used = None
        maxcore_used = None
        control_path = workspace_path / 'CONTROL.txt'
        if control_path.exists():
            pal_used, maxcore_used = parse_resource_settings(control_path.read_text())

        if pal_used is None or maxcore_used is None:
            inp_candidate = None
            try:
                if _calc_selected_inp_path and Path(_calc_selected_inp_path).exists():
                    inp_candidate = Path(_calc_selected_inp_path)
            except Exception:
                inp_candidate = None

            if inp_candidate is None:
                inp_files = sorted(workspace_path.glob('*.inp'))
                inp_candidate = inp_files[0] if inp_files else None

            if inp_candidate is not None and inp_candidate.exists():
                pal_inp, maxcore_inp = parse_inp_resources(inp_candidate.read_text())
                if pal_used is None:
                    pal_used = pal_inp
                if maxcore_used is None:
                    maxcore_used = maxcore_inp

        if pal_used is None or maxcore_used is None:
            calc_override_status.value = '<span style="color:#d32f2f;">PAL/maxcore not found in CONTROL.txt or .inp</span>'
            return

        mem_mb = pal_used * maxcore_used

        if is_recalc:
            # Build the sbatch command for --recalc
            job_name = f'recalc_{workspace_root.replace("/", "_")}'
            env_vars = f"DELFIN_MODE=delfin-recalc,DELFIN_JOB_NAME={job_name}"
        else:
            # Build the sbatch command for --occupier-override
            job_name = f'occ_{override_value.replace("=", "_")}'
            env_vars = f"DELFIN_MODE=delfin-recalc-override,DELFIN_JOB_NAME={job_name},DELFIN_OVERRIDE={override_value}"
        
        result = subprocess.run([
            'sbatch',
            f'--export=ALL,{env_vars}',
            f'--time={calc_override_time.value}',
            '--ntasks=1',
            f'--cpus-per-task={pal_used}',
            f'--mem={mem_mb}M',
            f'--job-name={job_name}',
            str(SUBMIT_TEMPLATES_DIR / 'submit_delfin.sh')
        ], cwd=str(workspace_path), capture_output=True, text=True)
        
        if result.returncode == 0:
            calc_override_status.value = f'<span style="color:#388e3c;">Submitted from {workspace_root}: {result.stdout.strip()}</span>'
        else:
            calc_override_status.value = f'<span style="color:#d32f2f;">Error: {result.stderr.strip()}</span>'
    except Exception as e:
        calc_override_status.value = f'<span style="color:#d32f2f;">Error: {e}</span>'

calc_options_dropdown.observe(calc_on_options_change, names='value')
calc_override_btn.on_click(calc_on_override_start)

def calc_update_path_display():
    global _calc_current_path
    display_path = '/' + _calc_current_path if _calc_current_path else '/'
    calc_path_label.value = f'<b>📂 Path:</b> <code style="background:#eee;padding:2px 5px; white-space:nowrap; overflow:hidden; text-overflow:ellipsis; display:block; max-width:100%;" title="{display_path}">{display_path}</code>'
    calc_back_btn.disabled = (_calc_current_path == '')

def calc_set_message(message):
    calc_content_area.value = (
        "<div style='height:520px; overflow-y:auto; border:1px solid #ddd; padding:6px;"
        " font-family:monospace; white-space:pre; background:#fafafa;'>"
        f"{html.escape(message)}"
        "</div>"
    )

def calc_clear_selection():
    calc_file_list.value = None
    calc_file_list.index = None


def calc_delete_hide_confirm():
    global _calc_delete_current
    calc_delete_confirm.layout.display = 'none'
    calc_delete_status.value = ''
    _calc_delete_current = False


def calc_on_delete_click(button):
    global _calc_delete_current
    if not calc_file_list.value or calc_file_list.value.startswith('('):
        # No selection: delete current folder
        if _calc_current_path:
            calc_delete_label.value = f'<b>Delete current folder?</b> <code>{html.escape(_calc_current_path)}</code>'
            _calc_delete_current = True
            calc_delete_confirm.layout.display = 'flex'
        else:
            calc_delete_status.value = '<span style="color:#d32f2f;">No item selected.</span>'
        return
    calc_delete_label.value = '<b>Delete selected item?</b>'
    _calc_delete_current = False
    calc_delete_confirm.layout.display = 'flex'


def calc_on_delete_yes(button):
    global _calc_current_path, _calc_delete_current
    delete_current = _calc_delete_current
    if delete_current:
        if not _calc_current_path:
            calc_delete_status.value = '<span style="color:#d32f2f;">No current folder.</span>'
            calc_delete_confirm.layout.display = 'none'
            _calc_delete_current = False
            return
        full_path = CALC_DIR / _calc_current_path
        name = _calc_current_path
    else:
        selected = calc_file_list.value
        if not selected or selected.startswith('('):
            calc_delete_status.value = '<span style="color:#d32f2f;">No item selected.</span>'
            calc_delete_confirm.layout.display = 'none'
            _calc_delete_current = False
            return
        name = selected[2:].strip()
        full_path = (CALC_DIR / _calc_current_path / name) if _calc_current_path else (CALC_DIR / name)
    try:
        if full_path.is_dir():
            shutil.rmtree(full_path)
        else:
            full_path.unlink()
        calc_delete_status.value = f'<span style="color:#2e7d32;">Deleted: {html.escape(name)}</span>'
    except Exception as exc:
        calc_delete_status.value = f'<span style="color:#d32f2f;">Delete failed: {html.escape(str(exc))}</span>'
    calc_delete_confirm.layout.display = 'none'
    _calc_delete_current = False
    if delete_current:
        _calc_current_path = _calc_current_path.rsplit('/', 1)[0] if '/' in _calc_current_path else ''
    # Preserve filter and refresh
    saved_filter = calc_folder_search.value
    calc_list_directory()
    if saved_filter:
        calc_folder_search.value = saved_filter
        calc_filter_file_list()
    calc_clear_selection()


def calc_on_delete_no(button):
    calc_delete_hide_confirm()


def calc_update_view():
    show_mol = calc_view_toggle.value
    if show_mol:
        calc_mol_container.layout.display = 'block'
        calc_content_area.layout.display = 'none'
        calc_edit_area.layout.display = 'none'
        calc_content_label.layout.display = 'none'
        if 'calc_content_toolbar' in globals():
            calc_content_toolbar.layout.display = 'none'
        if 'calc_recalc_toolbar' in globals():
            calc_recalc_toolbar.layout.display = 'none'
    else:
        calc_mol_container.layout.display = 'none'
        calc_content_label.layout.display = 'block'
        if 'calc_content_toolbar' in globals():
            calc_content_toolbar.layout.display = 'none' if _calc_recalc_active else 'flex'
        if _calc_recalc_active:
            calc_content_area.layout.display = 'none'
            calc_edit_area.layout.display = 'block'
            if 'calc_recalc_toolbar' in globals():
                calc_recalc_toolbar.layout.display = 'flex'
        else:
            calc_content_area.layout.display = 'block'
            calc_edit_area.layout.display = 'none'
            if 'calc_recalc_toolbar' in globals():
                calc_recalc_toolbar.layout.display = 'none'




def calc_reset_recalc_state():
    global _calc_selected_inp_path, _calc_selected_inp_base, _calc_recalc_active
    _calc_selected_inp_path = None
    _calc_selected_inp_base = ''
    _calc_recalc_active = False
    calc_recalc_btn.disabled = True
    calc_submit_recalc_btn.disabled = True
    calc_recalc_status.value = ''
    calc_edit_area.value = ''
    calc_edit_area.layout.display = 'none'
    if 'calc_recalc_toolbar' in globals():
        calc_recalc_toolbar.layout.display = 'none'
    calc_content_label.value = '<b>📄 File Content:</b>'


def calc_get_recalc_base_name(stem):
    match = re.match(r'^(.*)_recalc_(\d+)$', stem)
    return match.group(1) if match else stem


def calc_next_recalc_index(job_dir, base_name):
    pattern = re.compile(rf'^{re.escape(base_name)}_recalc_(\d+)\.(?:inp|out)$')
    max_idx = 0
    for entry in job_dir.iterdir():
        match = pattern.match(entry.name)
        if match:
            max_idx = max(max_idx, int(match.group(1)))
    return max_idx + 1


def calc_on_recalc_click(button):
    global _calc_recalc_active
    if not _calc_selected_inp_path or not _calc_selected_inp_path.exists():
        calc_recalc_status.value = '<span style="color:#d32f2f;">No .inp file selected.</span>'
        return
    try:
        content = _calc_selected_inp_path.read_text()
    except Exception as exc:
        calc_recalc_status.value = f'<span style="color:#d32f2f;">Error reading file: {html.escape(str(exc))}</span>'
        return
    calc_edit_area.value = content
    _calc_recalc_active = True
    calc_submit_recalc_btn.disabled = False
    calc_view_toggle.value = False
    calc_content_label.value = '<b>📄 File Content (editable):</b>'
    calc_update_view()


def calc_on_submit_recalc(button):
    if not _calc_selected_inp_path or not _calc_selected_inp_path.exists():
        calc_recalc_status.value = '<span style="color:#d32f2f;">No .inp file selected.</span>'
        return
    inp_content = calc_edit_area.value
    if not inp_content.strip():
        calc_recalc_status.value = '<span style="color:#d32f2f;">Input content is empty.</span>'
        return

    job_dir = _calc_selected_inp_path.parent
    base_stem = _calc_selected_inp_path.stem
    base_name = calc_get_recalc_base_name(base_stem)
    next_idx = calc_next_recalc_index(job_dir, base_name)
    job_name = f"{base_name}_recalc_{next_idx}"

    try:
        inp_path = job_dir / f"{job_name}.inp"
        inp_path.write_text(inp_content)

        slurm_time = calc_recalc_time.value.strip() or '24:00:00'
        
        # Parse resources from input
        pal_used, maxcore_used = parse_inp_resources(inp_content)
        if pal_used is None:
            pal_used = orca_pal.value
        if maxcore_used is None:
            maxcore_used = orca_maxcore.value
        total_mem = pal_used * maxcore_used

        result = submit_job(
            job_dir=job_dir,
            mode='orca',
            job_name=job_name,
            inp_file=f'{job_name}.inp',
            time_limit=slurm_time,
            ntasks=pal_used,
            mem_mb=total_mem
        )
    except Exception as exc:
        calc_recalc_status.value = f'<span style="color:#d32f2f;">Error: {html.escape(str(exc))}</span>'
        return

    if result.returncode == 0:
        job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '(unknown)'
        calc_recalc_status.value = (
            f'<span style="color:#2e7d32;">Submitted {html.escape(job_name)} (ID: {html.escape(job_id)})</span>'
        )
    else:
        msg = result.stderr or result.stdout or 'Unknown sbatch error'
        calc_recalc_status.value = f'<span style="color:#d32f2f;">{html.escape(msg)}</span>'

def calc_run_js(script):
    if not script:
        return
    display(Javascript(script))


def calc_scroll_to(target):
    if target == 'top':
        js = """
        setTimeout(function(){
            const box = document.getElementById('calc-content-box');
            if (box) { box.scrollTop = 0; }
        }, 0);
        """
        calc_run_js(js)
        return
    if target == 'bottom':
        js = """
        setTimeout(function(){
            const box = document.getElementById('calc-content-box');
            if (box) { box.scrollTop = box.scrollHeight; }
        }, 0);
        """
        calc_run_js(js)
        return
    if target == 'match' and _calc_current_match >= 0:
        js = """
        setTimeout(function(){
            const box = document.getElementById('calc-content-box');
            const el = document.getElementById('calc-current-match');
            if (!box || !el) return;
            const boxRect = box.getBoundingClientRect();
            const elRect = el.getBoundingClientRect();
            const delta = (elRect.top - boxRect.top) - (box.clientHeight / 2);
            box.scrollTop += delta;
        }, 0);
        """
        calc_run_js(js)




def calc_apply_highlight(query, current_index):
    if query is None:
        query = ''
    js = r"""
    (function() {
        const box = document.getElementById('calc-content-box');
        const el = document.getElementById('calc-content-text');
        if (!box || !el) return;
        const text = el.textContent || '';
        const q = __QUERY__;
        if (!q) {
            el.textContent = text;
            return;
        }
        function esc(s) {
            return s.replace(/[&<>"]/g, function(c) {
                return {'&':'&amp;','<':'&lt;','>':'&gt;','"':'&quot;'}[c];
            });
        }
        function escapeRegExp(s) {
            return s.replace(/[.*+?^${}()|[\\]\\]/g, '\\$&');
        }
        const re = new RegExp(escapeRegExp(q), 'gi');
        let html = '';
        let last = 0;
        let i = 0;
        let m;
        while ((m = re.exec(text)) !== null) {
            html += esc(text.slice(last, m.index));
            const cls = (i === __INDEX__) ? 'calc-match current' : 'calc-match';
            const id = (i === __INDEX__) ? 'calc-current-match' : '';
            html += `<mark class="${cls}" ${id ? 'id=\"' + id + '\"' : ''}>${esc(m[0])}</mark>`;
            last = m.index + m[0].length;
            i++;
        }
        html += esc(text.slice(last));
        el.innerHTML = html;
    })();
    """
    js = js.replace('__QUERY__', repr(query)).replace('__INDEX__', str(current_index))
    calc_run_js(js)


def calc_extract_orca_xyz_block(text):
    lines = text.splitlines()
    start_idx = None
    for i, line in enumerate(lines):
        if re.match(r'^\s*\*\s*xyz\b', line, flags=re.IGNORECASE):
            start_idx = i + 1
            break
    if start_idx is None:
        return None
    coords = []
    for line in lines[start_idx:]:
        if re.match(r'^\s*\*', line):
            break
        if not line.strip():
            continue
        coords.append(line.rstrip())
    return coords if coords else None


def calc_build_xyz_from_input(text, title='input.txt'):
    coords = calc_extract_orca_xyz_block(text)
    if coords:
        atom_count = len(coords)
        return f"{atom_count}\n{title}\n" + '\n'.join(coords)
    lines = [ln for ln in text.strip().splitlines() if ln.strip()]
    if not lines:
        return None
    # If first line is an integer atom count, assume header exists
    if re.fullmatch(r'\d+', lines[0].strip()):
        return '\n'.join(lines)
    atom_count = len(lines)
    return f"{atom_count}\n{title}\n" + '\n'.join(lines)



def calc_render_content(scroll_to=None):
    global _calc_file_content, _calc_search_spans, _calc_current_match
    if not _calc_file_content:
        calc_set_message('Select a file...')
        return

    text = _calc_file_content
    calc_content_area.value = (
        "<style>"
        ".calc-match { background: #fff59d; padding: 0 2px; }"
        ".calc-match.current { background: #ffcc80; }"
        "</style>"
        "<div id='calc-content-box' style='height:520px; overflow-y:auto; overflow-x:hidden; border:1px solid #ddd; padding:6px; background:#fafafa; width:100%; box-sizing:border-box;'>"
        "<div id='calc-content-text' style='white-space:pre-wrap; overflow-wrap:anywhere; word-break:break-word; font-family:monospace; font-size:12px; line-height:1.3;'>"
        f"{html.escape(text)}"
        "</div></div>"
    )
    if scroll_to:
        calc_scroll_to(scroll_to)
def calc_filter_file_list(change=None):
    """Filter file list based on search input."""
    global _calc_all_items
    query = calc_folder_search.value.strip().lower()
    if not query:
        calc_file_list.options = _calc_all_items
    else:
        filtered = [item for item in _calc_all_items if query in item.lower()]
        calc_file_list.options = filtered if filtered else ['(No matches)']

def calc_orca_terminated_normally(path):
    """Check for ORCA termination marker in the tail of a text file."""
    try:
        with path.open('rb') as f:
            f.seek(0, 2)
            size = f.tell()
            tail_size = 20000
            f.seek(max(0, size - tail_size))
            tail = f.read()
        text = tail.decode('utf-8', errors='ignore')
        return '****ORCA TERMINATED NORMALLY****' in text
    except Exception:
        return False

def calc_list_directory():
    global _calc_current_path, _calc_file_content, _calc_all_items, _calc_search_spans, _calc_current_match
    _calc_file_content = ''
    _calc_all_items = []
    _calc_search_spans = []
    _calc_current_match = -1
    calc_file_list.options = []
    calc_mol_viewer.clear_output()
    calc_mol_container.layout.display = 'none'
    calc_file_info.value = ''
    calc_search_result.value = ''
    calc_folder_search.value = ''
    calc_prev_btn.disabled = True
    calc_next_btn.disabled = True
    calc_view_toggle.value = False
    calc_view_toggle.disabled = True
    calc_update_view()

    calc_set_message('Select a file...')

    current_dir = CALC_DIR / _calc_current_path if _calc_current_path else CALC_DIR

    if not current_dir.exists():
        calc_file_list.options = ['(Folder not found)']
        return

    items = []
    try:
        entries = sorted(current_dir.iterdir(), key=lambda x: (not x.is_dir(), x.name.lower()))
        for entry in entries:
            if entry.is_dir():
                items.append(f'📁 {entry.name}')
            else:
                suffix = entry.suffix.lower()
                if suffix == '.xyz':
                    items.append(f'🔬 {entry.name}')
                elif suffix == '.png':
                    items.append(f'🖼 {entry.name}')
                elif suffix in ['.out', '.log']:
                    is_delfin = entry.name.lower().startswith('delfin_')
                    ok = True if is_delfin else calc_orca_terminated_normally(entry)
                    icon = '📄' if ok else '❌'
                    items.append(f'{icon} {entry.name}')
                elif suffix in ['.inp', '.sh']:
                    items.append(f'📝 {entry.name}')
                elif suffix in ['.cube', '.cub']:
                    items.append(f'🧊 {entry.name}')
                elif suffix in ['.gbw', '.cis', '.densities']:
                    items.append(f'💾 {entry.name}')
                else:
                    items.append(f'📄 {entry.name}')
    except PermissionError:
        items = ['(Permission denied)']

    _calc_all_items = items if items else ['(Empty folder)']
    calc_file_list.options = _calc_all_items
    calc_update_path_display()

def calc_on_back(b):
    global _calc_current_path
    if _calc_current_path:
        parts = _calc_current_path.split('/')
        _calc_current_path = '/'.join(parts[:-1])
        calc_list_directory()

def calc_on_home(b):
    global _calc_current_path
    _calc_current_path = ''
    calc_list_directory()

def calc_on_refresh(b):
    calc_list_directory()

def calc_go_top(b):
    """Scroll to beginning of file."""
    if _calc_file_content:
        calc_render_content(scroll_to='top')

def calc_go_bottom(b):
    """Scroll to end of file."""
    if _calc_file_content:
        calc_render_content(scroll_to='bottom')

def calc_pos_to_line_col(pos):
    if pos < 0:
        return 0, 0
    line = _calc_file_content.count('\n', 0, pos) + 1
    last_nl = _calc_file_content.rfind('\n', 0, pos)
    col = pos + 1 if last_nl == -1 else pos - last_nl
    return line, col

def calc_update_nav_buttons():
    if not _calc_search_spans:
        calc_prev_btn.disabled = True
        calc_next_btn.disabled = True
        return
    calc_prev_btn.disabled = (_calc_current_match <= 0)
    calc_next_btn.disabled = (_calc_current_match >= len(_calc_search_spans) - 1)

def calc_update_search_result():
    if not _calc_search_spans:
        calc_search_result.value = ''
        return
    if _calc_current_match < 0:
        calc_search_result.value = f'<span style="color:green;">{len(_calc_search_spans)} matches</span>'
        return
    start, _ = _calc_search_spans[_calc_current_match]
    line, col = calc_pos_to_line_col(start)
    calc_search_result.value = (
        f'<b>{_calc_current_match + 1}/{len(_calc_search_spans)}</b> '
        f'<span style="color:#555;">(line {line}, col {col})</span>'
    )

def calc_do_search(b=None):
    """Search in file content."""
    global _calc_file_content, _calc_search_spans, _calc_current_match
    query = calc_search_input.value.strip()
    # If input empty but dropdown has value, use dropdown
    if not query and calc_search_suggest.value and calc_search_suggest.value != '(Select)':
        query = calc_search_suggest.value
        calc_search_input.value = query
    _calc_search_spans = []
    _calc_current_match = -1

    if not query or not _calc_file_content:
        calc_search_result.value = ''
        calc_update_nav_buttons()
        calc_apply_highlight('', -1)
        return

    pattern = re.compile(re.escape(query), re.IGNORECASE)
    _calc_search_spans = [match.span() for match in pattern.finditer(_calc_file_content)]

    if not _calc_search_spans:
        calc_search_result.value = '<span style="color:red;">0 matches</span>'
        calc_update_nav_buttons()
        calc_apply_highlight('', -1)
        return

    _calc_current_match = 0
    calc_update_nav_buttons()
    calc_update_search_result()
    calc_apply_highlight(query, _calc_current_match)
    calc_scroll_to('match')



def calc_on_suggest(change):
    value = change['new']
    if not value or value == '(Select)':
        return
    calc_search_input.value = value

def calc_show_match():
    """Display current match and scroll to it."""
    if not _calc_search_spans or _calc_current_match < 0:
        return
    calc_update_nav_buttons()
    calc_update_search_result()
    calc_apply_highlight(calc_search_input.value.strip(), _calc_current_match)
    calc_scroll_to('match')

def calc_prev_match(b):
    global _calc_current_match, _calc_search_spans
    if _calc_search_spans and _calc_current_match > 0:
        _calc_current_match -= 1
        calc_show_match()

def calc_next_match(b):
    global _calc_current_match, _calc_search_spans
    if _calc_search_spans and _calc_current_match < len(_calc_search_spans) - 1:
        _calc_current_match += 1
        calc_show_match()

def calc_on_select(change):
    global _calc_current_path, _calc_file_content, _calc_search_spans, _calc_current_match, _calc_selected_inp_path, _calc_selected_inp_base, _calc_xyz_frames, _calc_xyz_current_frame
    selected = change['new']
    if not selected or selected.startswith('('):
        return

    name = selected[2:].strip()
    full_path = (CALC_DIR / _calc_current_path / name) if _calc_current_path else (CALC_DIR / name)

    # Navigate into directory
    if full_path.is_dir():
        _calc_current_path = f'{_calc_current_path}/{name}' if _calc_current_path else name
        calc_list_directory()
        return

    # Reset search state
    _calc_search_spans = []
    _calc_current_match = -1
    calc_search_result.value = ''
    calc_search_input.value = ''
    calc_update_nav_buttons()
    calc_delete_hide_confirm()
    calc_update_options_dropdown()

    # Show file
    calc_mol_viewer.clear_output()
    calc_mol_container.layout.display = 'none'
    calc_content_area.layout.display = 'block'
    calc_view_toggle.value = False
    calc_view_toggle.disabled = True
    calc_file_info.value = ''
    _calc_file_content = ''
    calc_reset_recalc_state()
    # Reset xyz trajectory controls (using module-level globals)
    _calc_xyz_frames.clear()
    _calc_xyz_current_frame[0] = 0
    calc_xyz_controls.layout.display = 'none'
    calc_coord_controls.layout.display = 'none'
    calc_xyz_frame_label.value = ''
    calc_xyz_frame_input.value = 1
    calc_xyz_frame_input.max = 1
    calc_xyz_frame_total.value = '<b>/ 1</b>'
    

    if not full_path.exists():
        calc_set_message('File not found')
        return

    # File info
    size = full_path.stat().st_size
    size_str = f'{size/(1024*1024):.2f} MB' if size > 1024*1024 else (f'{size/1024:.2f} KB' if size > 1024 else f'{size} B')

    suffix = full_path.suffix.lower()
    name_lower = full_path.name.lower()

    # Turbomole coord: show molecule viewer
    if name_lower == 'coord':
        calc_view_toggle.disabled = False
        calc_view_toggle.value = False
        calc_update_view()
        try:
            content = full_path.read_text()
            _calc_file_content = content
            xyz_data = coord_to_xyz(content)
            if xyz_data:
                calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, Turbomole coord)'
                _calc_xyz_frames.clear()
                _calc_xyz_frames.append(xyz_data)
                _calc_xyz_current_frame[0] = 0
                calc_xyz_controls.layout.display = 'none'
                calc_coord_controls.layout.display = 'flex'
                with calc_mol_viewer:
                    clear_output(wait=True)
                    view = py3Dmol.view(width=520, height=520)
                    view.addModel(xyz_data, 'xyz')
                    view.setStyle({}, {'stick': {'radius': 0.15}, 'sphere': {'scale': 0.25}})
                    view.zoomTo()
                    view.show()
            else:
                calc_file_info.value = f'<b>{html.escape(name)}</b> (could not parse)'
        except Exception as e:
            calc_file_info.value = f'<b>{html.escape(name)}</b> (error: {e})'
        return

    # XYZ: show molecule viewer (with trajectory support)
    if suffix == '.xyz':
        calc_view_toggle.disabled = False
        calc_view_toggle.value = False
        calc_update_view()
        try:
            content = full_path.read_text()
            _calc_file_content = content
            lines = content.split('\n')
            
            # Parse multi-frame XYZ
            _calc_xyz_frames.clear()
            _calc_xyz_frames.extend(parse_xyz_frames(content))
            n_frames = len(_calc_xyz_frames)
            
            if n_frames > 1:
                # Multi-frame trajectory
                calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, {n_frames} frames)'
                _calc_xyz_current_frame[0] = 0
                calc_xyz_frame_input.max = n_frames
                calc_xyz_frame_input.value = 1
                calc_xyz_frame_total.value = f"<b>/ {n_frames}</b>"
                calc_xyz_controls.layout.display = 'flex'
                calc_coord_controls.layout.display = 'none'
                calc_update_xyz_viewer(initial_load=True)
            else:
                # Single frame
                calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, {len(lines)} lines)'
                _calc_xyz_frames.clear()
                _calc_xyz_frames.extend(parse_xyz_frames(content))
                if _calc_xyz_frames:
                    _calc_xyz_current_frame[0] = 0
                    calc_xyz_frame_input.max = len(_calc_xyz_frames)
                    calc_xyz_frame_input.value = 1
                    calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                    calc_xyz_controls.layout.display = 'flex'
                    calc_coord_controls.layout.display = 'none'
                else:
                    calc_xyz_controls.layout.display = 'none'
                calc_xyz_frame_label.value = ''
                with calc_mol_viewer:
                    view = py3Dmol.view(width=520, height=520)
                    view.addModel(content, 'xyz')
                    view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                    view.zoomTo()
                    view.show()
            calc_render_content(scroll_to='top')
        except Exception as e:
            calc_set_message(f'Error: {e}')

    # PNG image
    elif suffix == '.png':
        try:
            data = full_path.read_bytes()
            b64 = base64.b64encode(data).decode('ascii')
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            calc_content_area.value = (
                "<div style='border:1px solid #ddd; padding:6px; background:#fafafa;'>"
                f"<img src='data:image/png;base64,{b64}' style='max-width:50%; max-height:520px; height:auto; display:block;' />"
                "</div>"
            )
            calc_update_view()
        except Exception as e:
            calc_set_message(f'Error: {e}')

    # Cube volumetric data
    elif suffix in ['.cube', '.cub']:
        calc_view_toggle.disabled = False
        calc_view_toggle.value = False
        calc_update_view()
        try:
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            lower_name = name.lower()
            if lower_name.endswith('.esp.cube') or lower_name.endswith('.esp.cub'):
                from delfin.reporting.esp_report import generate_esp_png_for_state

                workspace_root = None
                search = full_path.parent
                while True:
                    if (search / 'CONTROL.txt').exists():
                        workspace_root = search
                        break
                    if search.parent == search:
                        break
                    search = search.parent
                if workspace_root is None:
                    workspace_root = full_path.parent

                rendered = generate_esp_png_for_state(workspace_root, 'S0')

                calc_set_message('ESP rendered via DELFIN report pipeline. Toggle Visualize to view.')
                _calc_file_content = ''
                with calc_mol_viewer:
                    if rendered and rendered.exists():
                        data = rendered.read_bytes()
                        b64 = base64.b64encode(data).decode('ascii')
                        display(HTML(f"<img src='data:image/png;base64,{b64}' style='max-width:100%; max-height:520px; height:auto; display:block;' />"))
                    else:
                        print('ESP render failed (no PNG generated).')
            else:
                content = full_path.read_text()
                _calc_file_content = ''
                calc_set_message('Cube file loaded. Toggle Visualize to render isosurfaces.')
                with calc_mol_viewer:
                    view = py3Dmol.view(width=520, height=520)
                    view.addModel(content, 'cube')
                    view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                    view.addVolumetricData(content, 'cube', {'isoval': 0.02, 'color': '#0026ff', 'opacity': 0.85})
                    view.addVolumetricData(content, 'cube', {'isoval': -0.02, 'color': '#b00010', 'opacity': 0.85})
                    view.zoomTo()
                    view.show()
        except Exception as e:
            calc_set_message(f'Error: {e}')

    # Binary files
    elif suffix in ['.gbw', '.cis', '.densities', '.tmp']:
        calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
        calc_set_message(f'Binary file ({size_str})\n\nCannot display binary content.')

    # DOCX (not previewed)
    elif suffix == '.docx':
        calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
        calc_set_message('DOCX preview not supported. Download/open locally.')

    # Text files - load entire file
    else:
        try:
            content = full_path.read_text()
            _calc_file_content = content
            lines = content.split('\n')
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, {len(lines)} lines)'
            calc_render_content(scroll_to='top')
            if name == 'input.txt':
                calc_view_toggle.disabled = False
                calc_view_toggle.value = False
                calc_update_view()
                try:
                    xyz_content = calc_build_xyz_from_input(content, title=name)
                    if xyz_content:
                        _calc_xyz_frames.clear()
                        _calc_xyz_frames.extend(parse_xyz_frames(xyz_content))
                        if _calc_xyz_frames:
                            _calc_xyz_current_frame[0] = 0
                            calc_xyz_frame_input.max = len(_calc_xyz_frames)
                            calc_xyz_frame_input.value = 1
                            calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                            calc_xyz_controls.layout.display = 'flex'
                            calc_coord_controls.layout.display = 'none'
                        else:
                            calc_xyz_controls.layout.display = 'none'
                        calc_xyz_frame_label.value = ''
                        _calc_xyz_frames.clear()
                        _calc_xyz_frames.extend(parse_xyz_frames(xyz_content))
                        if _calc_xyz_frames:
                            _calc_xyz_current_frame[0] = 0
                            calc_xyz_frame_input.max = len(_calc_xyz_frames)
                            calc_xyz_frame_input.value = 1
                            calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                            calc_xyz_controls.layout.display = 'flex'
                            calc_coord_controls.layout.display = 'none'
                        else:
                            calc_xyz_controls.layout.display = 'none'
                        calc_xyz_frame_label.value = ''
                        with calc_mol_viewer:
                            view = py3Dmol.view(width=520, height=520)
                            view.addModel(xyz_content, 'xyz')
                            view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                            view.zoomTo()
                            view.show()
                except Exception as exc:
                    calc_set_message(f'Error: {exc}')
            elif suffix == '.inp':
                try:
                    xyz_content = calc_build_xyz_from_input(content, title=name)
                    if xyz_content:
                        _calc_xyz_frames.clear()
                        _calc_xyz_frames.extend(parse_xyz_frames(xyz_content))
                        if _calc_xyz_frames:
                            _calc_xyz_current_frame[0] = 0
                            calc_xyz_frame_input.max = len(_calc_xyz_frames)
                            calc_xyz_frame_input.value = 1
                            calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                            calc_xyz_controls.layout.display = 'flex'
                            calc_coord_controls.layout.display = 'none'
                        else:
                            calc_xyz_controls.layout.display = 'none'
                        calc_xyz_frame_label.value = ''
                        calc_view_toggle.disabled = False
                        calc_view_toggle.value = False
                        calc_update_view()
                        with calc_mol_viewer:
                            view = py3Dmol.view(width=520, height=520)
                            view.addModel(xyz_content, 'xyz')
                            view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                            view.zoomTo()
                            view.show()
                except Exception as exc:
                    calc_set_message(f'Error: {exc}')
            if suffix == '.inp':
                _calc_selected_inp_path = full_path
                _calc_selected_inp_base = full_path.stem
                calc_recalc_btn.disabled = False
            calc_update_view()
        except UnicodeDecodeError:
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            calc_set_message(f'Binary file ({size_str})\n\nCannot display.')
        except Exception as e:
            calc_set_message(f'Error: {e}')

# Connect event handlers
calc_back_btn.on_click(calc_on_back)
calc_home_btn.on_click(calc_on_home)
calc_refresh_btn.on_click(calc_on_refresh)
calc_top_btn.on_click(calc_go_top)
calc_bottom_btn.on_click(calc_go_bottom)
calc_search_btn.on_click(calc_do_search)
calc_search_input.observe(lambda change: calc_do_search(), names='value')
calc_search_suggest.observe(calc_on_suggest, names='value')
calc_view_toggle.observe(lambda change: calc_update_view(), names='value')
calc_prev_btn.on_click(calc_prev_match)
calc_next_btn.on_click(calc_next_match)
calc_recalc_btn.on_click(calc_on_recalc_click)
calc_submit_recalc_btn.on_click(calc_on_submit_recalc)
calc_delete_btn.on_click(calc_on_delete_click)
calc_delete_yes_btn.on_click(calc_on_delete_yes)
calc_delete_no_btn.on_click(calc_on_delete_no)
calc_file_list.observe(calc_on_select, names='value')
calc_folder_search.observe(calc_filter_file_list, names='value')

# Initialize
if CALC_DIR.exists():
    calc_list_directory()
else:
    calc_file_list.options = ['(calc folder not found)']

# Build layout
calc_mol_container = widgets.VBox([
    calc_mol_label,
    calc_xyz_frame_label,
    calc_xyz_controls,
    calc_coord_controls,
    calc_mol_viewer
], layout=widgets.Layout(display='none', margin='0 0 10px 0', width='100%', align_items='flex-start'))

calc_nav_bar = widgets.VBox([
    calc_path_label,
    widgets.HBox([calc_back_btn, calc_home_btn, calc_refresh_btn, calc_delete_btn], layout=widgets.Layout(width='100%', overflow_x='hidden', justify_content='flex-start', gap='6px'))
], layout=widgets.Layout(width='100%', overflow_x='hidden'))

calc_content_toolbar = widgets.HBox([
    calc_top_btn, calc_bottom_btn,
    widgets.HTML('&nbsp;│&nbsp;'),
    calc_search_input, calc_search_suggest, calc_search_btn, widgets.HTML('&nbsp;&nbsp;'),
    calc_prev_btn, calc_next_btn, calc_options_dropdown, calc_override_input, calc_override_time, calc_override_btn, calc_search_result
], layout=widgets.Layout(margin='5px 0', width='100%', overflow_x='hidden', gap='6px'))

calc_recalc_toolbar = widgets.HBox([
    calc_recalc_time,
    calc_submit_recalc_btn,
    calc_recalc_status
], layout=widgets.Layout(margin='5px 0', width='100%', overflow_x='hidden', gap='8px', align_items='center'))
calc_recalc_toolbar.layout.display = 'none'

calc_left = widgets.VBox([
    calc_nav_bar,
    calc_folder_search,
    calc_file_list
], layout=widgets.Layout(flex='0 0 260px', min_width='220px', max_width='300px', padding='5px', overflow_x='hidden', overflow_y='hidden'))

calc_css = widgets.HTML(
    '<style>'
    '#calc-content-box { overflow-x:hidden !important; }'
    '.calc-tab, .calc-tab * { overflow-x:hidden !important; box-sizing:border-box; }.calc-tab { overflow:hidden !important; }.calc-right { overflow:hidden !important; }.calc-tab .widget-vbox, .calc-tab .widget-hbox { overflow-y:hidden !important; }.calc-right .widget-output, .calc-right .jupyter-widgets-output-area { overflow:hidden !important; }.calc-right .widget-output .output_area { overflow:hidden !important; }.calc-right .widget-output .output { overflow:hidden !important; }.calc-right .output_scroll, .calc-right .output_subarea, .calc-right .output_wrapper { overflow:hidden !important; }.calc-right .jp-OutputArea, .calc-right .jp-OutputArea-child, .calc-right .jp-OutputArea-output { overflow:hidden !important; }.calc-right .p-Widget { overflow:hidden !important; }.calc-right .widget-textarea { overflow:visible !important; }.calc-right .widget-textarea textarea { overflow-y:auto !important; }'
    '.calc-left, .calc-right { overflow-x:hidden !important; }'
    '.calc-left * { max-width:100% !important; }'
    '.calc-right * { max-width:100% !important; }'
    '.calc-left .widget-select, .calc-left .widget-select select { overflow-x:hidden !important; }'
    '.calc-left .widget-select select { text-overflow: ellipsis; white-space: nowrap; }'
    '.calc-left .widget-text input { overflow-x:hidden !important; overflow-y:hidden !important; text-overflow: ellipsis; }'
    '.calc-right .widget-text input { overflow-x:hidden !important; overflow-y:hidden !important; }'
    '.calc-tab .widget-text { overflow:visible !important; }'
    '.calc-tab input { overflow:hidden !important; height:26px !important; line-height:26px !important; padding:0 6px !important; box-sizing:border-box !important; }'
    '.calc-tab input::-webkit-scrollbar { width:0; height:0; display:none; }'
    '.calc-tab input { scrollbar-width:none; }'
    '.calc-left .widget-vbox { overflow:hidden !important; }'
    '.calc-left code { display:block !important; overflow:hidden !important; text-overflow:ellipsis !important; white-space:nowrap !important; }'
    '</style>'
)

calc_right = widgets.VBox([
    widgets.HBox([
        calc_file_info,
        widgets.HBox([calc_view_toggle], layout=widgets.Layout(gap='6px'))
    ], layout=widgets.Layout(align_items='center', justify_content='space-between')),
    calc_mol_container,
    calc_content_label,
    calc_delete_confirm,
    calc_delete_status,
    calc_recalc_toolbar,
    calc_content_toolbar,
    calc_override_status,
    calc_content_area,
    calc_edit_area
], layout=widgets.Layout(flex='1 1 0', min_width='0', padding='5px', overflow_x='hidden', overflow_y='hidden'))

tab5_content = widgets.VBox([
    calc_css,
    widgets.HTML('<h3>📂 Calculations Browser</h3>'),
    widgets.HBox([calc_left, calc_right], layout=widgets.Layout(width='100%', overflow_x='hidden', align_items='stretch', gap='16px'))
], layout=widgets.Layout(padding='10px', overflow_x='hidden', width='100%', max_width='100%', height='780px'))
tab5_content.add_class('calc-tab')
calc_left.add_class('calc-left')
calc_right.add_class('calc-right')



# === Repo Update (git pull) ===
busy_css = widgets.HTML(
    "<style>"
    ".delfin-busy { display:inline-block; width:14px; height:14px; border:2px solid #1976d2; border-top-color: transparent; border-radius:50%; animation: delfin-spin 0.9s linear infinite; }"
    "@keyframes delfin-spin { from { transform: rotate(0deg); } to { transform: rotate(360deg); } }"
    ".widget-textarea textarea { resize: none !important; }"
    "textarea { resize: none !important; }"
    "</style>"
)

busy_indicator = widgets.HTML(value="")

def set_delfin_busy(is_busy):
    busy_indicator.value = "<span class='delfin-busy' title='Working'></span>" if is_busy else ""

pull_delfin_btn = widgets.Button(
    description='PULL DELFIN',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

orca_version_dropdown = widgets.Dropdown(
    options=[(p.name.replace('_linux_x86-64_shared_openmpi418_avx2',''), str(p)) for p in _orca_candidates if ('orca_6_1_1' in p.name or 'orca_6_1_0' in p.name)] if _orca_candidates else [('ORCA not found', '')],
    value=ORCA_BASE_SELECTED if ORCA_BASE_SELECTED else '',
    description='ORCA:',
    layout=widgets.Layout(width='170px'),
    style={'description_width': 'initial'},
    disabled=(not _orca_candidates)
)

def _update_orca_base(change):
    global ORCA_BASE_SELECTED
    if change.get('new'):
        ORCA_BASE_SELECTED = change['new']

orca_version_dropdown.observe(_update_orca_base, names='value')

pull_delfin_output = widgets.Output()

# Prefer detected repo root; fallback to ROOT_DIR/software/delfin
_repo_dir = _deflin_root if _deflin_root else (ROOT_DIR / 'software' / 'delfin')

def handle_pull_delfin(button):
    with pull_delfin_output:
        clear_output()
        if not _repo_dir or not Path(_repo_dir).exists():
            print(f'Repo path not found: {_repo_dir}')
            return
        print(f'Running git pull in {_repo_dir} ...')
        set_delfin_busy(True)
        try:
            result = subprocess.run(
                ['git', '-C', str(_repo_dir), 'pull'],
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                check=False
            )
            print(result.stdout.strip() or '(no output)')
        except Exception as e:
            print(f'Error: {e}')
        finally:
            set_delfin_busy(False)

pull_delfin_btn.on_click(handle_pull_delfin)



# === TURBOMOLE Input Builder ===
# Interactive define panel for TURBOMOLE job setup

import threading
import queue
import time as _time

tm_module_options = ['ridft', 'dscf', 'ricc2', 'escf', 'grad', 'rdgrad', 'rigrad', 'jobex', 'jobex-noopt', 'aoforce', 'NumForce', 'freeh', 'ccsdf12', 'pnoccsd']
tm_para_options = ['SMP', 'MPI']

tm_job_name = widgets.Text(value='', placeholder='Enter job name (required)', description='Job Name:', layout=widgets.Layout(width='400px'))
tm_coords = widgets.Textarea(value='', placeholder='Paste XYZ or TURBOMOLE coord format', description='Coordinates:', layout=widgets.Layout(width='90%', height='180px', overflow_y='auto'))
tm_convert_smiles_btn = widgets.Button(description='CONVERT SMILES', button_style='info', icon='exchange', layout=widgets.Layout(width='150px'))
tm_module = widgets.Dropdown(options=tm_module_options, value='ridft', description='TM Module:', layout=widgets.Layout(width='200px'))
tm_para_arch = widgets.Dropdown(options=tm_para_options, value='SMP', description='Parallel:', layout=widgets.Layout(width='150px'))
tm_nprocs = widgets.IntText(value=40, description='CPUs:', layout=widgets.Layout(width='150px'))
tm_memory = widgets.IntText(value=6000, description='Mem/CPU (MB):', layout=widgets.Layout(width='180px'))
tm_slurm_time = widgets.Text(value='48:00:00', description='Time Limit:', layout=widgets.Layout(width='180px'))

tm_define_output = widgets.Output(layout=widgets.Layout(width='90%', height='400px', border='1px solid #333', overflow='auto', padding='5px', flex='0 0 320px'))
tm_define_input = widgets.Textarea(value='', placeholder='Type response, press Enter', layout=widgets.Layout(width='90%', height='32px'), continuous_update=True)
tm_define_send = widgets.Button(description='Send', button_style='primary', layout=widgets.Layout(width='150px'))
tm_start_define_btn = widgets.Button(description='Start define', button_style='info', icon='terminal', layout=widgets.Layout(width='150px'))
tm_stop_define_btn = widgets.Button(description='Stop define', button_style='danger', icon='stop', layout=widgets.Layout(width='150px'), disabled=True)
tm_save_btn = widgets.Button(description='Save to Folder', button_style='warning', icon='save', layout=widgets.Layout(width='150px'))
tm_submit_btn = widgets.Button(description='SUBMIT JOB', button_style='success', icon='rocket', layout=widgets.Layout(width='150px'))
tm_status = widgets.HTML(value='')
tm_load_path = widgets.Text(value='', placeholder='Job name or path', description='Load from:', layout=widgets.Layout(width='350px'))
tm_load_btn = widgets.Button(description='Load', button_style='info', icon='upload', layout=widgets.Layout(width='100px'))
tm_replay_btn = widgets.Button(description='Replay', button_style='warning', icon='play', layout=widgets.Layout(width='100px'))
tm_mol_output = widgets.Output(layout=widgets.Layout(width='600px', height='450px', border='2px solid #4CAF50', overflow='hidden'))

def _tm_append_to_json(cmd):
    """Append a define command to delfin_import.json"""
    if _tm_job_dir is None:
        return
    json_file = _tm_job_dir / 'delfin_import.json'
    try:
        if json_file.exists():
            with open(json_file) as f:
                data = json.load(f)
        else:
            data = {'define_commands': [], 'settings': {}}
        data['define_commands'].append(cmd)
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=2)
    except:
        pass

_tm_process = None
_tm_child = None
_tm_job_dir = None
_tm_output_queue = queue.Queue()
_tm_running = False
_tm_ready = False
_tm_sending = False
_tm_last_sent = None
_tm_last_sent_at = 0.0
_tm_input_enabled = False
_tm_empty_block_until = 0.0
_tm_last_edit_at = 0.0
_tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}

def xyz_to_coord(xyz_text):
    global _tm_smiles_cache
    raw = xyz_text.strip()
    if not raw:
        _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
        return raw, None
    lines = raw.split('\n')
    if lines[0].strip().startswith('$coord'):
        _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
        return raw, None
    note = None
    # Allow SMILES input by converting it to XYZ first.
    try:
        if is_smiles(raw):
            xyz_string, num_atoms, method, error = smiles_to_xyz(raw)
            if error or not xyz_string:
                _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
                return raw, f"SMILES conversion failed: {error}"
            raw = f"{num_atoms}\nConverted from SMILES ({method})\n{xyz_string}"
            lines = raw.split('\n')
            note = f"SMILES converted via {method}"
            _tm_smiles_cache = {'smiles': xyz_text.strip(), 'xyz': xyz_string, 'method': method}
        else:
            _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
    except Exception as e:
        _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
        return raw, f"SMILES conversion error: {e}"
    coord_lines = []
    for line in lines:
        parts = line.split()
        if len(parts) >= 4:
            try:
                elem = parts[0]
                if not elem[0].isalpha(): continue
                x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
                bohr = 1.8897259886
                coord_lines.append(f"  {x*bohr:14.8f}  {y*bohr:14.8f}  {z*bohr:14.8f}  {elem.lower()}")
            except: continue
    if not coord_lines:
        # Fallback: if nothing parsed and it looks like a single-line SMILES,
        # try SMILES conversion even if detection failed.
        if len(lines) == 1 and ' ' not in raw and '\t' not in raw:
            try:
                xyz_string, num_atoms, method, error = smiles_to_xyz(raw)
                if error or not xyz_string:
                    _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
                    return raw, f"SMILES conversion failed: {error}"
                bohr = 1.8897259886
                coord_lines = []
                for line in xyz_string.split('\n'):
                    parts = line.split()
                    if len(parts) >= 4:
                        try:
                            elem = parts[0]
                            x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
                            coord_lines.append(f"  {x*bohr:14.8f}  {y*bohr:14.8f}  {z*bohr:14.8f}  {elem.lower()}")
                        except Exception:
                            continue
                if coord_lines:
                    _tm_smiles_cache = {'smiles': raw, 'xyz': xyz_string, 'method': method}
                    coord_text = "$coord\n" + "\n".join(coord_lines) + "\n$end"
                    return coord_text, f"SMILES converted via {method}"
            except Exception as e:
                _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
                return raw, f"SMILES conversion error: {e}"
        _tm_smiles_cache = {'smiles': None, 'xyz': None, 'method': None}
        return raw, note
    coord_text = "$coord\n" + "\n".join(coord_lines) + "\n$end"
    return coord_text, note

def handle_tm_convert_smiles(button=None):
    raw = tm_coords.value.strip()
    if not raw:
        tm_status.value = '<span style="color:red;">Coordinates/SMILES required!</span>'
        return
    if not is_smiles(raw):
        tm_status.value = '<span style="color:orange;">Input is not a SMILES string.</span>'
        return
    tm_status.value = '<span style="color:orange;">Converting SMILES...</span>'
    xyz_string, num_atoms, method, error = smiles_to_xyz(raw)
    if error or not xyz_string:
        tm_status.value = f'<span style="color:red;">SMILES conversion failed: {error}</span>'
        return
    xyz_block = f"{num_atoms}\nConverted from SMILES ({method})\n{xyz_string}"
    coord_text, coord_note = xyz_to_coord(xyz_block)
    tm_coords.value = coord_text
    if coord_note:
        tm_status.value = f'<span style="color:green;">{coord_note}</span>'
    else:
        tm_status.value = f'<span style="color:green;">SMILES converted via {method}</span>'

def coord_to_xyz(coord_text):
    lines = coord_text.strip().split('\n')
    xyz_lines = []
    bohr_to_ang = 0.529177249
    in_coord = False
    for line in lines:
        if line.strip().startswith('$coord'): in_coord = True; continue
        if line.strip().startswith('$'): in_coord = False; continue
        if in_coord:
            parts = line.split()
            if len(parts) >= 4:
                try:
                    x, y, z = float(parts[0])*bohr_to_ang, float(parts[1])*bohr_to_ang, float(parts[2])*bohr_to_ang
                    xyz_lines.append(f"{parts[3].capitalize()}  {x:12.6f}  {y:12.6f}  {z:12.6f}")
                except: continue
    if not xyz_lines:
        for line in lines:
            parts = line.split()
            if len(parts) >= 4 and parts[0][0].isalpha():
                try: float(parts[1]); xyz_lines.append(line)
                except: continue
    if not xyz_lines: return None
    return f"{len(xyz_lines)}\nTM Preview\n" + "\n".join(xyz_lines)

def update_tm_molecule_view(change=None):
    with tm_mol_output:
        clear_output(wait=True)
        raw = tm_coords.value.strip()
        if not raw: print("No coordinates."); return
        xyz = coord_to_xyz(raw)
        if not xyz:
            lines = [l for l in raw.split('\n') if len(l.split())>=4 and l.split()[0][0].isalpha()]
            xyz = f"{len(lines)}\nPreview\n" + "\n".join(lines) if lines else None
        if not xyz: print("Could not parse."); return
        try:
            import py3Dmol
            v = py3Dmol.view(width=580, height=430)
            v.addModel(xyz, "xyz")
            v.setStyle({'stick': {}, 'sphere': {'radius': 0.3}})
            v.zoomTo(); v.show()
        except Exception as e: print(f"Error: {e}")

def _tm_reader_thread(proc, q):
    """Legacy reader thread; no-op when using pexpect."""
    return

def _tm_scroll_to_bottom():
    """Scroll terminal output to bottom."""
    from IPython.display import display, Javascript
    js = Javascript('''
        setTimeout(function() {
            var outputs = document.querySelectorAll('.widget-output');
            outputs.forEach(function(out) {
                if (out.closest('.widget-vbox')) {
                    out.scrollTop = out.scrollHeight;
                }
            });
        }, 100);
    ''')
    display(js)

def _tm_update_output():
    """Update output widget from pexpect child (non-blocking)."""
    global _tm_running, _tm_ready, _tm_input_enabled, _tm_child
    updated = False
    if _tm_child is None:
        return
    try:
        import pexpect
        import re
        while True:
            try:
                chunk = _tm_child.read_nonblocking(size=4096, timeout=0)
            except pexpect.TIMEOUT:
                break
            except pexpect.EOF:
                with tm_define_output:
                    print("\n[define finished]")
                _tm_running = False
                _tm_ready = False
                _tm_input_enabled = False
                tm_start_define_btn.disabled = False
                tm_stop_define_btn.disabled = True
                tm_define_input.disabled = True
                tm_define_send.disabled = True
                break
            if not chunk:
                break
            with tm_define_output:
                clean = re.sub(r'\x1b\[[0-9;]*[a-zA-Z]', '', chunk)
                clean = clean.replace('\r\n', '\n').replace('\r', '')
                print(clean, end='', flush=True)
            updated = True
            _tm_ready = True
            if not _tm_input_enabled:
                tm_define_input.disabled = False
                tm_define_send.disabled = False
                _tm_input_enabled = True
                try:
                    from IPython.display import display, Javascript
                    display(Javascript("""
                        setTimeout(function () {
                            var el = document.querySelector('textarea[placeholder="Type response, press Enter"]');
                            if (el) { el.focus(); }
                        }, 50);
                    """))
                except Exception:
                    pass
        if (not _tm_child.isalive()) and _tm_running:
            with tm_define_output:
                print("\n[define finished]")
            _tm_running = False
            _tm_ready = False
            _tm_input_enabled = False
            tm_start_define_btn.disabled = False
            tm_stop_define_btn.disabled = True
            tm_define_input.disabled = True
            tm_define_send.disabled = True
            updated = True
    except Exception as e:
        with tm_define_output:
            print(f"[Output error: {e}]")
    if updated:
        _tm_scroll_to_bottom()

def handle_start_define(button):
    global _tm_process, _tm_child, _tm_job_dir, _tm_running, _tm_ready, _tm_input_enabled
    
    job_name = tm_job_name.value.strip().lstrip('/')
    if not job_name:
        tm_status.value = '<span style="color:red;">Job name required!</span>'
        return
    if not tm_coords.value.strip():
        tm_status.value = '<span style="color:red;">Coordinates required!</span>'
        return
    
    _tm_job_dir = BASE_DIR / job_name
    _tm_job_dir.mkdir(parents=True, exist_ok=True)
    # Initialize delfin_import.json for this job
    with open(_tm_job_dir / 'delfin_import.json', 'w') as f:
        json.dump({'define_commands': [], 'settings': {}}, f, indent=2)
    coord_text, coord_note = xyz_to_coord(tm_coords.value.strip())
    (_tm_job_dir / "coord").write_text(coord_text)
    
    with tm_define_output:
        clear_output()
        print(f"Job directory: {_tm_job_dir}")
        print("Starting define... (this may take a moment)")
        print("=" * 50)
    
    tm_status.value = f'<span style="color:green;">Dir: {_tm_job_dir}</span>'
    if coord_note:
        tm_status.value = f'<span style="color:green;">Dir: {_tm_job_dir} · {coord_note}</span>'
        with tm_define_output:
            print(coord_note)
    
    try:
        import pexpect
        cmd = 'source /etc/profile && module load chem/turbomole/7.9 2>&1 && define'
        _tm_child = pexpect.spawn('/bin/bash', ['-lc', cmd], cwd=str(_tm_job_dir), encoding='utf-8', echo=False)
        _tm_process = _tm_child
        
        _tm_running = True
        _tm_ready = False
        _tm_input_enabled = False
        _tm_empty_block_until = _time.time() + 2.0
        _tm_last_edit_at = _time.time()
        tm_start_define_btn.disabled = True
        tm_stop_define_btn.disabled = False
        tm_define_input.disabled = True
        tm_define_send.disabled = True
        
        # Read initial output - wait for define to start and show header
        for _ in range(20):
            _time.sleep(0.2)
            _tm_update_output()
        
        # Schedule periodic output updates
        def periodic_update():
            if _tm_running:
                _tm_update_output()
                # Re-schedule
                import asyncio
                try:
                    loop = asyncio.get_event_loop()
                    loop.call_later(0.2, periodic_update)
                except:
                    pass
        
        import asyncio
        try:
            loop = asyncio.get_event_loop()
            loop.call_later(0.5, periodic_update)
        except:
            pass
            
    except Exception as e:
        tm_status.value = f'<span style="color:red;">Error: {e}</span>'
        with tm_define_output:
            print(f"Error starting define: {e}")
            import traceback
            traceback.print_exc()

def handle_stop_define(button):
    global _tm_process, _tm_child, _tm_running, _tm_ready, _tm_input_enabled, _tm_empty_block_until
    
    _tm_running = False
    _tm_ready = False
    _tm_input_enabled = False
    _tm_empty_block_until = 0.0
    if _tm_child:
        try:
            _tm_child.close(force=True)
        except Exception:
            try:
                _tm_child.terminate(force=True)
            except Exception:
                pass
    _tm_child = None
    _tm_process = None
    
    tm_start_define_btn.disabled = False
    tm_stop_define_btn.disabled = True
    tm_define_input.disabled = True
    tm_define_send.disabled = True
    
    with tm_define_output:
        print("\n[define stopped]")

def handle_send_input(button=None, text_override=None):
    global _tm_process, _tm_child, _tm_sending, _tm_input_enabled
    
    if _tm_child is None or (hasattr(_tm_child, 'isalive') and not _tm_child.isalive()):
        with tm_define_output:
            print("[Process not running]")
        return
    # If define is still starting, poll output briefly instead of dropping Enter.
    if not _tm_input_enabled:
        for _ in range(20):
            _time.sleep(0.1)
            _tm_update_output()
            if _tm_input_enabled:
                break
    if _tm_sending:
        return
    
    _tm_sending = True
    # Allow an explicit text override from the Textarea enter handler.
    if text_override is None and button is None:
        _time.sleep(0.05)
    inp = tm_define_input.value if text_override is None else text_override
    if text_override is None:
        tm_define_input.value = ''
    
    try:
        if _tm_child is not None:
            # pexpect handles TTY semantics more reliably than os.write.
            _tm_child.sendline(inp)
            with tm_define_output:
                print(f">>> {inp}" if inp else ">>> [Enter]")
            _tm_append_to_json(inp)
            # Give define time to respond, then drain output.
            for _ in range(40):
                _time.sleep(0.1)
                _tm_update_output()
    except Exception as e:
        with tm_define_output:
            print(f"[Send error: {e}]")
    finally:
        _tm_sending = False

def _tm_mark_edit(change):
    global _tm_last_edit_at
    if change.get('name') == 'value':
        _tm_last_edit_at = _time.time()

def _tm_on_textarea_change(change):
    # Textarea receives a literal newline on Enter; use that as submit.
    if change.get('name') != 'value':
        return
    if _tm_sending or not _tm_input_enabled:
        return
    new = change.get('new', '') or ''
    if '\n' not in new:
        return
    # Take only the first line as the command, like a terminal.
    cmd = new.split('\n', 1)[0]
    tm_define_input.value = ''
    handle_send_input(text_override=cmd)

def handle_send_input_on_change(change):
    # Use observe for text inputs, but guard against recursive sends
    # when we clear the field programmatically in handle_send_input.
    if change.get('name') != 'value':
        return
    if _tm_sending:
        return
    if change.get('new') == change.get('old'):
        return
    handle_send_input()

def handle_tm_save(button):
    global _tm_job_dir
    job_name = tm_job_name.value.strip().lstrip('/')
    if not job_name:
        tm_status.value = '<span style="color:red;">Job name required!</span>'
        return
    _tm_job_dir = BASE_DIR / job_name
    if not (_tm_job_dir / "control").exists():
        tm_status.value = '<span style="color:orange;">No control file. Run define first!</span>'
        return
    tm_status.value = f'<span style="color:green;">Saved: {_tm_job_dir}</span>'

def handle_tm_load(button):
    """Load settings from a previous job's delfin_import.json"""
    load_path = tm_load_path.value.strip()
    # Handle relative paths (just job name) and absolute paths
    if not load_path.startswith('/'):
        load_dir = BASE_DIR / load_path
    else:
        load_dir = Path(load_path)
    json_file = load_dir / 'delfin_import.json'
    
    if not json_file.exists():
        tm_status.value = f'<span style="color:red;">No delfin_import.json in {load_dir}</span>'
        return
    
    try:
        with open(json_file) as f:
            data = json.load(f)
        
        settings = data.get('settings', {})
        if 'module' in settings:
            tm_module.value = settings['module']
        if 'para_arch' in settings:
            tm_para_arch.value = settings['para_arch']
        if 'nprocs' in settings:
            tm_nprocs.value = settings['nprocs']
        if 'memory_per_core' in settings:
            tm_memory.value = settings['memory_per_core']
        if 'slurm_time' in settings:
            tm_slurm_time.value = settings['slurm_time']
        
        # Load coords if available
        coord_file = load_dir / 'coord'
        if coord_file.exists():
            tm_coords.value = coord_file.read_text()
        
        # Show define commands
        cmds = data.get('define_commands', [])
        with tm_define_output:
            clear_output()
            if cmds:
                print("=== Define commands from loaded job ===")
                for cmd in cmds:
                    print(f"  {cmd}")
                print("=" * 40)
        
        tm_status.value = f'<span style="color:green;">Loaded from {load_dir.name}</span>'
    except Exception as e:
        tm_status.value = f'<span style="color:red;">Error: {e}</span>'

def handle_tm_load(button):
    global _tm_pending_replay
    if not tm_job_name.value.strip():
        tm_status.value = '<span style="color:red;">Enter Job Name first</span>'
        return
    if not tm_coords.value.strip():
        tm_status.value = '<span style="color:red;">Enter Coordinates first</span>'
        return
    load_path = tm_load_path.value.strip()
    if not load_path:
        tm_status.value = '<span style="color:red;">Enter path to load from</span>'
        return
    load_dir = BASE_DIR / load_path if not load_path.startswith('/') else Path(load_path)
    json_file = load_dir / 'delfin_import.json'
    if not json_file.exists():
        tm_status.value = f'<span style="color:red;">No delfin_import.json found</span>'
        return
    try:
        data = json.load(open(json_file))
        s = data.get('settings', {})
        if 'module' in s: tm_module.value = s['module']
        if 'para_arch' in s: tm_para_arch.value = s['para_arch']
        if 'nprocs' in s: tm_nprocs.value = s['nprocs']
        if 'memory_per_core' in s: tm_memory.value = s['memory_per_core']
        if 'slurm_time' in s: tm_slurm_time.value = s['slurm_time']
        _tm_pending_replay = data.get('define_commands', [])
        with tm_define_output:
            clear_output()
            print(f"Loaded {len(_tm_pending_replay)} commands. Start Define, then click Replay.")
        tm_status.value = f'<span style="color:green;">Loaded from {load_dir.name}</span>'
    except Exception as e:
        tm_status.value = f'<span style="color:red;">{e}</span>'

def handle_tm_replay(button):
    global _tm_pending_replay
    if not _tm_pending_replay:
        tm_status.value = '<span style="color:red;">No commands loaded</span>'
        return
    if _tm_child is None or not _tm_child.isalive():
        tm_status.value = '<span style="color:red;">Start Define first</span>'
        return
    cmds = _tm_pending_replay[:]
    _tm_pending_replay = []
    tm_status.value = f'<span style="color:orange;">Replaying {len(cmds)} commands...</span>'
    import time as _t
    
    for cmd in cmds:
        try:
            _tm_child.sendline(cmd)
            with tm_define_output:
                print(f">>> {cmd}")
            _tm_append_to_json(cmd)
            _t.sleep(3.0)  # Wait longer for define to process
            _tm_update_output()
        except:
            break
    tm_status.value = '<span style="color:green;">Replay done</span>'

def handle_tm_submit(button):
    global _tm_job_dir
    job_name = tm_job_name.value.strip().lstrip('/')
    if not job_name:
        tm_status.value = '<span style="color:red;">Job name required!</span>'
        return
    _tm_job_dir = BASE_DIR / job_name
    if not (_tm_job_dir / "control").exists():
        tm_status.value = '<span style="color:red;">No control file. Run define first!</span>'
        return
    if not (_tm_job_dir / "coord").exists():
        tm_status.value = '<span style="color:red;">No coord file!</span>'
        return
    mem_mb = tm_nprocs.value * tm_memory.value
    env_vars = f"TM_JOB_NAME={job_name},TM_MODULE={tm_module.value},TM_NPROCS={tm_nprocs.value},TM_PARA_ARCH={tm_para_arch.value}"
    try:
        result = subprocess.run(['sbatch', f'--export=ALL,{env_vars}', f'--time={tm_slurm_time.value}', '--ntasks=1', f'--cpus-per-task={tm_nprocs.value}', f'--mem={mem_mb}M', f'--job-name={job_name}', str(SUBMIT_TEMPLATES_DIR / 'submit_turbomole.sh')], cwd=str(_tm_job_dir), capture_output=True, text=True)
        if result.returncode == 0:
            # Save settings to delfin_import.json
            json_file = _tm_job_dir / 'delfin_import.json'
            try:
                if json_file.exists():
                    with open(json_file) as f:
                        data = json.load(f)
                else:
                    data = {'define_commands': [], 'settings': {}}
                data['settings'] = {
                    'module': tm_module.value,
                    'para_arch': tm_para_arch.value,
                    'nprocs': tm_nprocs.value,
                    'memory_per_core': tm_memory.value,
                    'slurm_time': tm_slurm_time.value,
                }
                with open(json_file, 'w') as f:
                    json.dump(data, f, indent=2)
            except:
                pass
            tm_status.value = f'<span style="color:green;">Submitted! {result.stdout.strip()}</span>'
            # Reset form
            tm_job_name.value = ''
            tm_coords.value = ''
            with tm_define_output:
                clear_output()
            with tm_mol_output:
                clear_output()
            tm_define_input.value = ''
            tm_define_input.disabled = True
            tm_define_send.disabled = True
        else:
            tm_status.value = f'<span style="color:red;">{result.stderr or result.stdout}</span>'
    except Exception as e:
        tm_status.value = f'<span style="color:red;">{e}</span>'

tm_start_define_btn.on_click(handle_start_define)
tm_stop_define_btn.on_click(handle_stop_define)
tm_convert_smiles_btn.on_click(handle_tm_convert_smiles)
tm_define_send.on_click(handle_send_input)
import warnings
warnings.filterwarnings('ignore', message='.*on_submit is deprecated.*', category=DeprecationWarning)
tm_define_input.observe(_tm_mark_edit, names='value')
tm_define_input.observe(_tm_on_textarea_change, names='value')
tm_save_btn.on_click(handle_tm_save)
tm_submit_btn.on_click(handle_tm_submit)
tm_load_btn.on_click(handle_tm_load)
tm_replay_btn.on_click(handle_tm_replay)
tm_load_btn.on_click(handle_tm_load)
tm_coords.observe(update_tm_molecule_view, names='value')
tm_define_input.disabled = True
tm_define_send.disabled = True
update_tm_molecule_view()

tm_input_row = widgets.HBox([tm_define_input])
tm_define_panel = widgets.VBox([
    widgets.HTML('<b>Interactive define Terminal:</b>'),
    tm_define_output,
    tm_input_row,
    widgets.HBox([tm_start_define_btn, tm_stop_define_btn], layout=widgets.Layout(gap='10px', margin='5px 0'))
], layout=widgets.Layout(flex='0 0 auto'))

tm_left = widgets.VBox([
    tm_job_name, tm_coords, widgets.HBox([tm_convert_smiles_btn], layout=widgets.Layout(width='90%')),
    widgets.HTML('<b>Molecule Preview:</b>'), tm_mol_output
], layout=widgets.Layout(width='44%', padding='10px', overflow_y='hidden'))

tm_right = widgets.VBox([
    tm_define_panel,
    widgets.HTML('<hr style="margin: 10px 0;">'),
    widgets.HTML('<b>Job Settings:</b>'),
    widgets.HBox([
        widgets.VBox([tm_module, tm_nprocs]),
        widgets.VBox([tm_para_arch, tm_memory])
    ], layout=widgets.Layout(gap='10px')),
    widgets.HBox([tm_slurm_time]),
    widgets.HBox([tm_load_path, tm_load_btn, tm_replay_btn], layout=widgets.Layout(gap='5px', margin='5px 0')),
    widgets.HBox([tm_save_btn, tm_submit_btn], layout=widgets.Layout(gap='10px', margin='10px 0')),
    tm_status
], layout=widgets.Layout(width='56%', padding='10px'))

tab6_content = widgets.VBox([
    widgets.HTML('<h3>TURBOMOLE Builder</h3>'),
    widgets.HTML('<a href="https://wiki.bwhpc.de/e/Turbomole" target="_blank" rel="noopener noreferrer">bwHPC Wiki</a> | <a href="https://www.turbomole.org/turbomole/turbomole-documentation/" target="_blank" rel="noopener noreferrer">TURBOMOLE Manuals</a>'),
    widgets.HBox([tm_left, tm_right])
], layout=widgets.Layout(padding='10px'))


# === Create Tabs ===
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content, tab4_content, tab6_content, tab5_content])
tabs.set_title(0, 'Submit Job')
tabs.set_title(1, 'Recalc')
tabs.set_title(2, 'Job Status')
tabs.set_title(3, 'ORCA Builder')
tabs.set_title(4, 'TURBOMOLE Builder')
tabs.set_title(5, 'Calculations')

display(widgets.VBox([
    busy_css,
    widgets.HBox([
        widgets.HTML('<h2 style=\"color:#1976d2; margin:0;\">DELFIN Dashboard</h2>'),
        widgets.HBox([busy_indicator, pull_delfin_btn, orca_version_dropdown], layout=widgets.Layout(margin='0 0 0 12px', align_items='center', gap='8px')),
    ], layout=widgets.Layout(align_items='center', justify_content='space-between', width='100%')),
    pull_delfin_output
], layout=widgets.Layout(width='100%')))
display(tabs)





























HTML(value='<h2 style="color:#1976d2;">DELFIN Dashboard</h2>')